In [79]:
import pyodbc 
import pandas as pd
import keyring
import getpass
import logging
import numpy as np

print("Ingrese usuario")
usuario = keyring.set_password('almacen', 'usuario', getpass.getpass())

print("Ingrese clave")
clave = keyring.set_password('almacen', 'clave', getpass.getpass())

'''
Otros posibles nombres de servidores pueden ser
servidor = 'localhost\sqlexpress' # para casos específicos
servidor = 'miservidor,puerto' # para especificar un puerto alterno
'''
servidor = '10.0.201.2'  # Nombre del servidor SQL con el cual se hará la conexión
bddatos = 'S1085f1a'  # Nombre de la base de datos SQL

try:
    connection = pyodbc.connect(
        driver='{iSeries Access ODBC Driver}',
        system=servidor,
        uid= keyring.get_password('almacen', 'usuario'),
        pwd=keyring.get_password('almacen', 'clave'))
    print("Conexión establecida")
except pyodbc.OperationalError as err:
    logging.warning(f"{err}\nVerifique conexión VPN")
except pyodbc.DataError as err:
    logging.warning(err)
except pyodbc.IntegrityError as err:
    logging.warning(err)
except pyodbc.ProgrammingError as err:
    logging.warning(err)
except pyodbc.NotSupportedError as err:
    logging.warning(err)
except pyodbc.DatabaseError as err:
    logging.warning(err)
except pyodbc.Error as err:
    logging.warning(err.args[1])

Ingrese usuario


 ····


Ingrese clave


 ··········


Conexión establecida


In [41]:
maquery = "with pagares2021 as (select concred_codcli, count(distinct periodo) meses_credito, count(periodo) pagares from modelos.base_vigentes where periodo >= 202104 group by concred_codcli having count(distinct periodo) = 8 and count(distinct periodo) = count(periodo) order by 1), temp as (select a.codcli, digits(b.servic) || digits(b.moneda) || digits(b.numope) pagare, b.tasade, (case b.TIFEPA when 0 then 'Frecuencia Fija' else 'Día Fijo Mensual' end) Modalidad_pago, (case b.plapag when 5 then 'Cuota Variable' else 'Cuota Fija' end) Tipo_plan_pagos , (case b.PAGAUT when 0 then 'No es LA' else 'Si es LA' end) LA, (select count(*) from siafd.f7101 d where d.codcli = a.codcli and d.flgdoc = 0 and d.situac = 1) pagares, c.meses_credito, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 4 AND CODEMP = 103 AND CTACBE LIKE '14_1%') deuda_abr21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 5 AND CODEMP = 103 AND CTACBE LIKE '14_1%') deuda_may21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 6 AND CODEMP = 103 AND CTACBE LIKE '14_1%') deuda_jun21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 7 AND CODEMP = 103 AND CTACBE LIKE '14_1%') deuda_jul21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 8 AND CODEMP = 103 AND CTACBE LIKE '14_1%') deuda_ago21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 9 AND CODEMP = 103 AND CTACBE LIKE '14_1%') deuda_sep21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 10 AND CODEMP = 103 AND CTACBE LIKE '14_1%') deuda_oct21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 4 AND CODEMP = 103 AND CTACBE LIKE '14_8%') dev_abr21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 5 AND CODEMP = 103 AND CTACBE LIKE '14_8%') dev_may21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 6 AND CODEMP = 103 AND CTACBE LIKE '14_8%') dev_jun21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 7 AND CODEMP = 103 AND CTACBE LIKE '14_8%') dev_jul21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 8 AND CODEMP = 103 AND CTACBE LIKE '14_8%') dev_ago21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 9 AND CODEMP = 103 AND CTACBE LIKE '14_8%') dev_sep21, (select sum(d.saldeu) from sofiah.a5811 d where d.coddeu = a.codsbs and d.AÑOREP = 2021 AND MESREP = 10 AND CODEMP = 103 AND CTACBE LIKE '14_8%') dev_oct21 from siafd.f5101 a left join siafd.f7101 b on a.codcli = b.codcli join pagares2021 c on a.codcli = c.concred_codcli where b.flgdoc = 0 and b.situac = 1 and b.AÑODES = 2021 AND b.mesdes = 04 and b.aÑOcan = 0) select * from temp where pagares = 1 and tasade > 0 and deuda_abr21 > deuda_may21 and deuda_may21 > deuda_jun21 and deuda_jun21 > deuda_jul21 and deuda_jul21 > deuda_ago21 and deuda_ago21 > deuda_sep21 and deuda_sep21 > deuda_oct21"
16182478,127936200,188595413

In [2]:
query = """
with pagares2021 as (select concred_codcli, count(distinct periodo) meses_credito, count(periodo) pagares
                     from modelos.base_vigentes
                     where periodo >= 202104 and periodo <=202111
                     group by concred_codcli
                     having count(distinct periodo) = 8
                        and count(distinct periodo) = count(periodo)
                     order by 1),
     temp as (select a.codcli,
                     digits(b.servic) || digits(b.moneda) || digits(b.numope)                  pagare,
                     b.tasade,
                     (case b.TIFEPA when 0 then 'Frecuencia Fija' else 'Día Fijo Mensual' end) Modalidad_pago,
                     (case b.plapag when 5 then 'Cuota Variable' else 'Cuota Fija' end)        Tipo_plan_pagos,
                     (case b.PAGAUT when 0 then 'No es LA' else 'Si es LA' end)                LA,
                     (select count(*)
                      from siafd.f7101 d
                      where d.codcli = a.codcli and d.flgdoc = 0 and d.situac = 1)             pagares,
                     c.meses_credito,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 4
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_1%')                                               deuda_abr21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 5
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_1%')                                               deuda_may21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 6
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_1%')                                               deuda_jun21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 7
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_1%')                                               deuda_jul21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 8
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_1%')                                               deuda_ago21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 9
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_1%')                                               deuda_sep21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 10
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_1%')                                               deuda_oct21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 4
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_8%')                                               dev_abr21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 5
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_8%')                                               dev_may21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 6
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_8%')                                               dev_jun21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 7
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_8%')                                               dev_jul21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 8
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_8%')                                               dev_ago21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 9
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_8%')                                               dev_sep21,
                     (select sum(d.saldeu)
                      from sofiah.a5811 d
                      where d.coddeu = a.codsbs
                        and d.AÑOREP = 2021
                        AND MESREP = 10
                        AND CODEMP = 103
                        AND CTACBE LIKE '14_8%')                                               dev_oct21
              from siafd.f5101 a
                       left join siafd.f7101 b on a.codcli = b.codcli
                       join pagares2021 c on a.codcli = c.concred_codcli
              where b.flgdoc = 0
                and b.situac = 1
                and b.AÑODES = 2021
                AND b.mesdes = 04
                and b.aÑOcan = 0)
select *
from temp
where pagares = 1
  and tasade > 0
  and deuda_abr21 > deuda_may21
  and deuda_may21 > deuda_jun21
  and deuda_jun21 > deuda_jul21
  and deuda_jul21 > deuda_ago21
  and deuda_ago21 > deuda_sep21
  and deuda_sep21 > deuda_oct21
"""

In [4]:
df = pd.read_sql_query(query,connection)
df.head()

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,DEUDA_AGO21,DEUDA_SEP21,DEUDA_OCT21,DEV_ABR21,DEV_MAY21,DEV_JUN21,DEV_JUL21,DEV_AGO21,DEV_SEP21,DEV_OCT21
0,26260582.0,080019976972,30.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9747.14,...,9017.58,8765.19,8499.86,58.47,78.45,62.65,74.57,72.58,64.11,68.41
1,4215125.0,077012272390,26.82,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9534.57,...,8092.26,7592.74,7082.46,132.87,133.07,120.47,119.76,112.94,100.89,94.11
2,24088847.0,077012269280,40.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3744.43,...,2844.16,2523.08,2192.61,83.10,56.42,48.76,47.46,40.16,35.62,33.04
3,24182380.0,080019916301,32.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2785.29,...,2100.60,1859.23,1612.02,39.59,34.58,29.82,28.96,24.44,21.63,20.01
4,22191616.0,080019919812,55.21,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1919.91,...,1666.01,1575.35,1479.07,41.95,42.67,36.34,38.96,37.03,33.04,32.87


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CODCLI           2517 non-null   float64
 1   PAGARE           2517 non-null   object 
 2   TASADE           2517 non-null   float64
 3   MODALIDAD_PAGO   2517 non-null   object 
 4   TIPO_PLAN_PAGOS  2517 non-null   object 
 5   LA               2517 non-null   object 
 6   PAGARES          2517 non-null   int64  
 7   MESES_CREDITO    2517 non-null   int64  
 8   DEUDA_ABR21      2517 non-null   float64
 9   DEUDA_MAY21      2517 non-null   float64
 10  DEUDA_JUN21      2517 non-null   float64
 11  DEUDA_JUL21      2517 non-null   float64
 12  DEUDA_AGO21      2517 non-null   float64
 13  DEUDA_SEP21      2517 non-null   float64
 14  DEUDA_OCT21      2517 non-null   float64
 15  DEV_ABR21        2486 non-null   float64
 16  DEV_MAY21        2430 non-null   float64
 17  DEV_JUN21     

In [8]:
df.describe()

,CODCLI,TASADE,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,DEUDA_JUN21,DEUDA_JUL21,DEUDA_AGO21,DEUDA_SEP21,DEUDA_OCT21,DEV_ABR21,DEV_MAY21,DEV_JUN21,DEV_JUL21,DEV_AGO21,DEV_SEP21,DEV_OCT21
count,2.517000e+03,2517.000000,2517.0,2517.0,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2486.000000,2430.000000,2429.000000,2433.000000,2417.000000,2409.000000,2388.000000
mean,1.952681e+07,42.111931,1.0,8.0,9998.626397,9588.465232,9160.105224,8747.322058,8338.946992,7922.739515,7483.715836,107.030961,108.665399,96.356566,98.303547,92.547365,82.618518,80.682391
std,7.049458e+06,14.206282,0.0,0.0,21758.257683,21213.174647,20642.363564,20203.916983,19794.440375,19401.605916,19013.851929,143.764870,146.784309,133.118848,139.546368,138.659648,130.086263,132.242104
min,2.363000e+03,8.500000,1.0,8.0,180.000000,179.690000,179.370000,179.340000,178.620000,170.880000,156.320000,0.830000,1.580000,0.930000,0.400000,1.000000,0.380000,0.280000
25%,1.541280e+07,33.010000,1.0,8.0,2000.000000,1864.330000,1724.560000,1577.790000,1427.970000,1272.090000,1109.110000,33.660000,33.805000,28.730000,28.570000,25.630000,21.670000,20.027500
50%,2.222756e+07,43.000000,1.0,8.0,4000.000000,3717.510000,3430.290000,3130.470000,2841.020000,2537.230000,2217.640000,62.125000,61.275000,52.980000,52.340000,47.430000,40.650000,37.860000
75%,2.535434e+07,50.100000,1.0,8.0,10000.000000,9110.690000,8269.100000,7530.000000,6923.680000,6185.040000,5499.220000,117.290000,118.647500,103.730000,103.630000,97.640000,85.070000,82.165000
max,2.629792e+07,89.000000,1.0,8.0,350000.000000,343139.820000,337756.000000,331932.210000,326292.510000,320588.180000,314699.440000,1745.400000,1787.590000,1749.510000,1832.750000,1840.370000,1915.570000,1998.780000


In [113]:
#Importante: reemplazamos por 0 los valores nulos
dataset = df.fillna(0)

In [114]:
dataset.head()

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,DEUDA_AGO21,DEUDA_SEP21,DEUDA_OCT21,DEV_ABR21,DEV_MAY21,DEV_JUN21,DEV_JUL21,DEV_AGO21,DEV_SEP21,DEV_OCT21
0,26260582.0,080019976972,30.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9747.14,...,9017.58,8765.19,8499.86,58.47,78.45,62.65,74.57,72.58,64.11,68.41
1,4215125.0,077012272390,26.82,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9534.57,...,8092.26,7592.74,7082.46,132.87,133.07,120.47,119.76,112.94,100.89,94.11
2,24088847.0,077012269280,40.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3744.43,...,2844.16,2523.08,2192.61,83.10,56.42,48.76,47.46,40.16,35.62,33.04
3,24182380.0,080019916301,32.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2785.29,...,2100.60,1859.23,1612.02,39.59,34.58,29.82,28.96,24.44,21.63,20.01
4,22191616.0,080019919812,55.21,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1919.91,...,1666.01,1575.35,1479.07,41.95,42.67,36.34,38.96,37.03,33.04,32.87


In [115]:
dataset.describe()

,CODCLI,TASADE,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,DEUDA_JUN21,DEUDA_JUL21,DEUDA_AGO21,DEUDA_SEP21,DEUDA_OCT21,DEV_ABR21,DEV_MAY21,DEV_JUN21,DEV_JUL21,DEV_AGO21,DEV_SEP21,DEV_OCT21
count,2.517000e+03,2517.000000,2517.0,2517.0,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000
mean,1.952681e+07,42.111931,1.0,8.0,9998.626397,9588.465232,9160.105224,8747.322058,8338.946992,7922.739515,7483.715836,105.712741,104.909384,92.987723,95.022857,88.870473,79.073504,76.547298
std,7.049458e+06,14.206282,0.0,0.0,21758.257683,21213.174647,20642.363564,20203.916983,19794.440375,19401.605916,19013.851929,143.363483,145.584378,131.962926,138.329010,137.073797,128.360677,130.030724
min,2.363000e+03,8.500000,1.0,8.0,180.000000,179.690000,179.370000,179.340000,178.620000,170.880000,156.320000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.541280e+07,33.010000,1.0,8.0,2000.000000,1864.330000,1724.560000,1577.790000,1427.970000,1272.090000,1109.110000,32.850000,31.820000,26.560000,26.890000,24.200000,19.660000,17.860000
50%,2.222756e+07,43.000000,1.0,8.0,4000.000000,3717.510000,3430.290000,3130.470000,2841.020000,2537.230000,2217.640000,61.370000,58.910000,50.850000,50.630000,45.720000,38.530000,35.800000
75%,2.535434e+07,50.100000,1.0,8.0,10000.000000,9110.690000,8269.100000,7530.000000,6923.680000,6185.040000,5499.220000,116.370000,113.680000,100.150000,100.790000,93.230000,81.370000,77.270000
max,2.629792e+07,89.000000,1.0,8.0,350000.000000,343139.820000,337756.000000,331932.210000,326292.510000,320588.180000,314699.440000,1745.400000,1787.590000,1749.510000,1832.750000,1840.370000,1915.570000,1998.780000


In [35]:
#Fórmula general para calcular la fecha de pago estimada la cual obtiene como parametros:
#-Saldo en el periodo 1
#-Interes devengado en el periodo 1
#-Numero de días que tiene el mes del periodo 1
#-Saldo en el periodo 2
#-Interes devengado en el periodo 2
#-Numero de días que tiene el mes del periodo 2
import numpy as np

def get_fpago (s1,d1,N1,s2,d2,N2):
    if s1 == 0 or s2 ==0:
        return 0
    else:
        return np.round((np.log(d2/s2 + 1)*N1 - np.log(d1/s1 + 1)*N2)/(np.log(d2/s2 + 1)-np.log(d1/s1 + 1)),0)
    

In [117]:
#Calculamos la fecha de pago entre los periodos que tenemos disponibles (por ahora en duro)
dataset['fpago_abr_may'] = dataset.apply(lambda x : get_fpago(x['DEUDA_ABR21'],x['DEV_ABR21'],30,x['DEUDA_MAY21'],x['DEV_MAY21'],31),axis=1)
dataset['fpago_may_jun'] = dataset.apply(lambda x : get_fpago(x['DEUDA_MAY21'],x['DEV_MAY21'],31,x['DEUDA_JUN21'],x['DEV_JUN21'],30),axis=1)
dataset['fpago_jun_jul'] = dataset.apply(lambda x : get_fpago(x['DEUDA_JUN21'],x['DEV_JUN21'],30,x['DEUDA_JUL21'],x['DEV_JUL21'],31),axis=1)
dataset['fpago_jul_ago'] = dataset.apply(lambda x : get_fpago(x['DEUDA_JUL21'],x['DEV_JUL21'],31,x['DEUDA_AGO21'],x['DEV_AGO21'],31),axis=1)
dataset['fpago_ago_sep'] = dataset.apply(lambda x : get_fpago(x['DEUDA_AGO21'],x['DEV_AGO21'],31,x['DEUDA_SEP21'],x['DEV_SEP21'],30),axis=1)
dataset['fpago_sep_oct'] = dataset.apply(lambda x : get_fpago(x['DEUDA_SEP21'],x['DEV_SEP21'],30,x['DEUDA_OCT21'],x['DEV_OCT21'],31),axis=1)

<ipython-input-116-3922dcbbfe89>:11: RuntimeWarning: invalid value encountered in double_scalars
  return np.round((np.log(d2/s2 + 1)*N1 - np.log(d1/s1 + 1)*N2)/(np.log(d2/s2 + 1)-np.log(d1/s1 + 1)),0)


In [118]:
#Importante reemplazamos por 0 los valores nulos
dataset = dataset.fillna(0) 

In [119]:
dataset[dataset["CODCLI"]==26194059]

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,DEV_JUL21,DEV_AGO21,DEV_SEP21,DEV_OCT21,fpago_abr_may,fpago_may_jun,fpago_jun_jul,fpago_jul_ago,fpago_ago_sep,fpago_sep_oct
1984,26194059.0,080019871022,43.41,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1859.5,...,41.34,37.32,30.63,28.86,5.0,5.0,5.0,31.0,18.0,18.0


In [120]:
dataset.tail(10)

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,DEV_JUL21,DEV_AGO21,DEV_SEP21,DEV_OCT21,fpago_abr_may,fpago_may_jun,fpago_jun_jul,fpago_jul_ago,fpago_ago_sep,fpago_sep_oct
2507,23474336.0,080019896731,60.10,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1860.55,...,21.78,19.05,14.72,13.20,32.0,25.0,25.0,31.0,20.0,20.0
2508,23551276.0,077012266900,35.08,Día Fijo Mensual,Cuota Fija,No es LA,1,8,1500.0,1391.64,...,22.65,19.52,17.29,15.02,8.0,8.0,8.0,31.0,-6399.0,333394.0
2509,24460673.0,077012275830,43.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,5000.0,4786.92,...,82.82,78.46,66.18,69.24,12.0,12.0,12.0,31.0,22.0,21.0
2510,12133472.0,080019927962,49.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2500.0,2327.45,...,37.39,33.81,28.31,26.21,14.0,14.0,14.0,31.0,14.0,14.0
2511,19407330.0,073012128630,42.58,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2500.0,2323.89,...,40.93,36.94,31.25,27.21,10.0,10.0,10.0,31.0,10.0,-9826.0
2512,6660053.0,080019973532,60.10,Día Fijo Mensual,Cuota Fija,No es LA,1,8,1500.0,1424.39,...,0.00,0.00,0.00,0.00,31.0,0.0,0.0,0.0,0.0,0.0
2513,26207827.0,073012129200,45.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,1000.0,930.14,...,17.19,15.53,13.14,11.45,10.0,10.0,10.0,31.0,10.0,-12138.0
2514,24876802.0,071014209610,27.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2763.57,...,40.18,36.10,30.69,27.64,28.0,5.0,5.0,31.0,5.0,5.0
2515,24798402.0,073012127940,43.08,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3833.58,...,73.50,69.63,62.49,58.56,14571.0,10.0,10.0,31.0,10.0,6933.0
2516,24474003.0,080019909181,30.10,Día Fijo Mensual,Cuota Fija,No es LA,1,8,20000.0,19362.04,...,279.30,268.96,245.98,235.66,10.0,10.0,10.0,31.0,10.0,-209364.0


In [121]:
lista = [5,6,8,31,5,6]
listatrun = [x for i,x in enumerate(lista) if i!=3]
listatrun

[5, 6, 8, 5, 6]

In [122]:
#Función para encontrar la fecha de pago optima basada en 2 criterios:
#Elegir la fecha calculada que más se repite (Siempre y cuando sea más de la mitad)
#Elegir la fecha calculada mínima (Siempre y cuando sea mayor a 0)
from scipy import stats as st
def fpagocomun (*arguments):
    fechas = []
    for fecha in arguments:
        fechas.append(fecha)
    fechas_truncate = [x for i,x in enumerate(fechas) if i!=3] #Excluimos de encontrar la moda al indice 3 que corresponde al calculo entre julio-agosto
    moda=st.mode(fechas_truncate)
    valor_frecuente = moda[0].item(0)
    frecuencia = moda[1].item(0)
    #Calculamos el valor mínimo mayor a 0
    fehasmayores0 = list(filter(lambda x: x > 0, fechas))
    valor_minimo = min(fehasmayores0,default=None)    
    
    
    #Si una fecha de pago se repite al menos la mitad de veces se considerará como fecha de pago
    if frecuencia >= len (fechas)/2:
        indice_val_frecuente = [i for i, n in enumerate(fechas) if n==valor_frecuente][-1]
        return valor_frecuente ,indice_val_frecuente,"moda" #fechas.index(valor_frecuente)
    
    #De lo contrario se consideará el valor mínimo que sea mayor a 0
    else:
        indice_val_minimo = [i for i, n in enumerate(fechas) if n==valor_minimo][-1] if len(fehasmayores0)>0 else None
        return valor_minimo , indice_val_minimo,"minimo" #(fechas.index(valor_minimo) if len(fehasmayores0)>0 else None) #Por verificar
        #return None
    

In [123]:
lista = [4,5,4,5,6,3,6,2,2,2,3,2,2,2]
moda = st.mode(lista)
lista.index(2)
dato = None


In [124]:
list(filter(lambda c: 'fpago' in c, dataset.columns.tolist()))

['fpago_abr_may',
 'fpago_may_jun',
 'fpago_jun_jul',
 'fpago_jul_ago',
 'fpago_ago_sep',
 'fpago_sep_oct']

In [125]:
#Calculamos la fecha de pago óptima a partir de las fechas de pagos calculadas previamente
dataset['fe_pago_frecuente'] = dataset.apply(lambda x: fpagocomun(x['fpago_abr_may'],x['fpago_may_jun'],x['fpago_jun_jul'],x['fpago_jul_ago'],x['fpago_ago_sep'],x['fpago_sep_oct']),axis=1)

In [126]:
dataset.head()

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,DEV_AGO21,DEV_SEP21,DEV_OCT21,fpago_abr_may,fpago_may_jun,fpago_jun_jul,fpago_jul_ago,fpago_ago_sep,fpago_sep_oct,fe_pago_frecuente
0,26260582.0,080019976972,30.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9747.14,...,72.58,64.11,68.41,27.0,25.0,25.0,31.0,20.0,20.0,"(20.0, 5, minimo)"
1,4215125.0,077012272390,26.82,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9534.57,...,112.94,100.89,94.11,10.0,10.0,10.0,31.0,10.0,-213278.0,"(10.0, 4, moda)"
2,24088847.0,077012269280,40.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3744.43,...,40.16,35.62,33.04,34.0,15.0,15.0,31.0,-5665.0,15.0,"(15.0, 5, moda)"
3,24182380.0,080019916301,32.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2785.29,...,24.44,21.63,20.01,47.0,15.0,15.0,31.0,-12653.0,15.0,"(15.0, 5, moda)"
4,22191616.0,080019919812,55.21,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1919.91,...,37.03,33.04,32.87,13.0,22.0,22.0,31.0,13.0,13.0,"(13.0, 5, moda)"


In [127]:
#Optamos mejor por tener las fechas de pago calculadas en una sola columna dentro de una lista, al igual que la deuda y los intereses devengados, para facilitar su cálculo
dataset['lista_pago_frecuente'] = dataset[['fpago_abr_may','fpago_may_jun','fpago_jun_jul','fpago_jul_ago','fpago_ago_sep','fpago_sep_oct']].values.tolist()
dataset['lista_deuda'] = dataset.filter(like='DEUDA_', axis=1).values.tolist()
dataset['lista_dev'] = dataset.filter(like='DEV_', axis=1).values.tolist()

In [128]:
#Por ahora agregamos en duro el número de días que tiene cada mes
n_dias = [30,31,30,31,31,30,31]
dataset['n_dias'] = [n_dias for _ in range(len(df))]

In [129]:
dataset[dataset['fe_pago_frecuente'].isna()] #207,622
dataset[dataset['fpago_abr_may']<0].head(5)

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,fpago_may_jun,fpago_jun_jul,fpago_jul_ago,fpago_ago_sep,fpago_sep_oct,fe_pago_frecuente,lista_pago_frecuente,lista_deuda,lista_dev,n_dias
19,26205247.0,080019900841,19.50,Día Fijo Mensual,Cuota Fija,No es LA,1,8,20000.0,19037.89,...,10.0,10.0,31.0,10.0,98868.0,"(10.0, 4, moda)","[-994225.0, 10.0, 10.0, 31.0, 10.0, 98868.0]","[20000.0, 19037.89, 18060.06, 17056.67, 16046....","[208.92, 198.87, 179.63, 178.17, 167.62, 149.3...","[30, 31, 30, 31, 31, 30, 31]"
23,10450241.0,080019941411,42.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,5000.0,4651.66,...,17.0,17.0,31.0,17.0,6845.0,"(17.0, 4, moda)","[-23714.0, 17.0, 17.0, 31.0, 17.0, 6845.0]","[5000.0, 4651.66, 4292.36, 3917.33, 3534.92, 3...","[68.65, 63.87, 54.7, 53.79, 48.53, 40.02, 34.83]","[30, 31, 30, 31, 31, 30, 31]"
33,20499702.0,080019903091,42.30,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2791.40,...,10.0,10.0,31.0,10.0,15289.0,"(10.0, 4, moda)","[-8600.0, 10.0, 10.0, 31.0, 10.0, 15289.0]","[3000.0, 2791.4, 2576.18, 2351.4, 2122.22, 188...","[62.37, 58.04, 50.99, 48.89, 44.12, 37.32, 32.49]","[30, 31, 30, 31, 31, 30, 31]"
66,22867750.0,080019903402,45.08,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2843.44,...,10.0,10.0,31.0,10.0,11184.0,"(10.0, 4, moda)","[-11547.0, 10.0, 10.0, 31.0, 10.0, 11184.0]","[3000.0, 2843.44, 2681.65, 2511.53, 2338.66, 2...","[65.83, 62.4, 56.01, 55.11, 51.32, 45.12, 41.26]","[30, 31, 30, 31, 31, 30, 31]"
68,25800869.0,080019902571,16.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,135000.0,134158.84,...,-229752.0,9.0,31.0,9.0,9.0,"(9.0, 5, moda)","[-277022.0, -229752.0, 9.0, 31.0, 9.0, 9.0]","[135000.0, 134158.84, 133246.3, 132321.63, 131...","[1173.88, 1166.57, 1158.63, 1205.63, 1197.63, ...","[30, 31, 30, 31, 31, 30, 31]"


In [57]:
#Formula para obtener la tea calculada a partir de las fechas de pago seleccionadas como óptimas
def tea_mes_from_list (lista_Saldos,lista_devengados,lista_frec_y_pago,lista_N_dias):
    fpago, index,_ = lista_frec_y_pago
    d = lista_devengados[index+1]
    S = lista_Saldos[index+1]
    p = fpago
    N = lista_N_dias[index+1]
    if p == N or S ==0:
        return None
    else:
        return np.round(np.power(d/S + 1, 360/(N-p))-1,4)*100

In [131]:
#Calculamos la tea 
dataset['tea_frecuente'] = dataset.apply(lambda x : tea_mes_from_list(x['lista_deuda'],x['lista_dev'],x['fe_pago_frecuente'],x['n_dias']),axis=1)

In [132]:
dataset.head()

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,fpago_jun_jul,fpago_jul_ago,fpago_ago_sep,fpago_sep_oct,fe_pago_frecuente,lista_pago_frecuente,lista_deuda,lista_dev,n_dias,tea_frecuente
0,26260582.0,080019976972,30.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9747.14,...,25.0,31.0,20.0,20.0,"(20.0, 5, minimo)","[27.0, 25.0, 25.0, 31.0, 20.0, 20.0]","[10000.0, 9747.14, 9519.73, 9264.14, 9017.58, ...","[58.47, 78.45, 62.65, 74.57, 72.58, 64.11, 68.41]","[30, 31, 30, 31, 31, 30, 31]",30.00
1,4215125.0,077012272390,26.82,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9534.57,...,10.0,31.0,10.0,-213278.0,"(10.0, 4, moda)","[10.0, 10.0, 10.0, 31.0, 10.0, -213278.0]","[10000.0, 9534.57, 9066.13, 8581.24, 8092.26, ...","[132.87, 133.07, 120.47, 119.76, 112.94, 100.8...","[30, 31, 30, 31, 31, 30, 31]",26.82
2,24088847.0,077012269280,40.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3744.43,...,15.0,31.0,-5665.0,15.0,"(15.0, 5, moda)","[34.0, 15.0, 15.0, 31.0, -5665.0, 15.0]","[4000.0, 3744.43, 3453.36, 3150.08, 2844.16, 2...","[83.1, 56.42, 48.76, 47.46, 40.16, 35.62, 33.04]","[30, 31, 30, 31, 31, 30, 31]",40.01
3,24182380.0,080019916301,32.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2785.29,...,15.0,31.0,-12653.0,15.0,"(15.0, 5, moda)","[47.0, 15.0, 15.0, 31.0, -12653.0, 15.0]","[3000.0, 2785.29, 2562.75, 2332.55, 2100.6, 18...","[39.59, 34.58, 29.82, 28.96, 24.44, 21.63, 20.01]","[30, 31, 30, 31, 31, 30, 31]",31.99
4,22191616.0,080019919812,55.21,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1919.91,...,22.0,31.0,13.0,13.0,"(13.0, 5, moda)","[13.0, 22.0, 22.0, 31.0, 13.0, 13.0]","[2000.0, 1919.91, 1841.74, 1753.24, 1666.01, 1...","[41.95, 42.67, 36.34, 38.96, 37.03, 33.04, 32.87]","[30, 31, 30, 31, 31, 30, 31]",55.21


In [133]:
#Agrupamos las teas calculadas para poder calcular el porcentaje de acierto. considerando que una tea con +- 0.05 es correcta
dataset['dif_tea_frec'] = dataset['tea_frecuente'] - dataset['TASADE']
bins_tea=[-np.inf,-0.049,0.05,np.inf]
labels_tea=['< -0.05', '+- 0.05','> 0.05']
dataset['group_dif_tea_frec'] = pd.cut(dataset['dif_tea_frec'], bins = bins_tea, labels = labels_tea)

In [134]:
dataset['group_dif_tea_frec'].value_counts(normalize=True)

+- 0.05    0.905578
> 0.05     0.058566
< -0.05    0.035857
Name: group_dif_tea_frec, dtype: float64

In [135]:
dataset.groupby(['TIPO_PLAN_PAGOS','LA','group_dif_tea_frec']).agg({'CODCLI':'count'})

CODCLI
TIPO_PLAN_PAGOS LA       group_dif_tea_frec        
Cuota Fija      No es LA < -0.05                 86
                         +- 0.05               2218
                         > 0.05                 115
                Si es LA < -0.05                  4
                         +- 0.05                 50
                         > 0.05                  30
Cuota Variable  No es LA < -0.05                  0
                         +- 0.05                  5
                         > 0.05                   2
                Si es LA < -0.05                  0
                         +- 0.05                  0
                         > 0.05                   0

In [136]:
c =dataset.groupby(['TIPO_PLAN_PAGOS','LA','group_dif_tea_frec']).agg({'CODCLI':'count'})
c / c.groupby(level=1).sum()

CODCLI
TIPO_PLAN_PAGOS LA       group_dif_tea_frec          
Cuota Fija      No es LA < -0.05             0.035449
                         +- 0.05             0.914262
                         > 0.05              0.047403
                Si es LA < -0.05             0.047619
                         +- 0.05             0.595238
                         > 0.05              0.357143
Cuota Variable  No es LA < -0.05             0.000000
                         +- 0.05             0.002061
                         > 0.05              0.000824
                Si es LA < -0.05             0.000000
                         +- 0.05             0.000000
                         > 0.05              0.000000

In [137]:
dataset[dataset['group_dif_tea_frec']!='+- 0.05']

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,fpago_ago_sep,fpago_sep_oct,fe_pago_frecuente,lista_pago_frecuente,lista_deuda,lista_dev,n_dias,tea_frecuente,dif_tea_frec,group_dif_tea_frec
14,26026032.0,080019878121,21.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,45000.0,42807.50,...,18.0,18.0,"(18.0, 5, moda)","[18.0, 5.0, 5.0, 31.0, 18.0, 18.0]","[45000.0, 42807.5, 40627.71, 38387.14, 36130.5...","[575.51, 593.41, 541.39, 532.13, 500.85, 432.9...","[30, 31, 30, 31, 31, 30, 31]",46.41,25.41,> 0.05
15,21657123.0,080019873671,42.59,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2785.95,...,18.0,18.0,"(18.0, 5, moda)","[18.0, 5.0, 5.0, 31.0, 18.0, 18.0]","[3000.0, 2785.95, 2571.33, 2347.18, 2118.56, 1...","[71.81, 72.31, 64.14, 60.92, 54.99, 45.12, 42.5]","[30, 31, 30, 31, 31, 30, 31]",103.31,60.72,> 0.05
20,17135112.0,077012279680,52.10,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1861.15,...,13.0,24.0,"(22.0, 2, moda)","[22.0, 22.0, 22.0, 31.0, 13.0, 24.0]","[2000.0, 1861.15, 1724.31, 1575.78, 1425.91, 1...","[37.63, 39.44, 32.44, 33.39, 30.21, 24.66, 24.15]","[30, 31, 30, 31, 31, 30, 31]",131.34,79.24,> 0.05
24,14314768.0,080019876591,20.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,23000.0,22192.10,...,18.0,18.0,"(18.0, 5, moda)","[18.0, 5.0, 5.0, 31.0, 18.0, 18.0]","[23000.0, 22192.1, 21395.63, 20574.34, 19750.9...","[281.27, 294.15, 272.62, 272.71, 261.79, 231.4...","[30, 31, 30, 31, 31, 30, 31]",44.00,24.00,> 0.05
29,19370712.0,080019878052,42.60,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3879.84,...,18.0,18.0,"(18.0, 5, moda)","[18.0, 5.0, 5.0, 31.0, 18.0, 18.0]","[4000.0, 3879.84, 3764.19, 3640.93, 3517.67, 3...","[95.76, 100.72, 93.92, 94.52, 91.32, 81.26, 84...","[30, 31, 30, 31, 31, 30, 31]",103.35,60.75,> 0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,14037586.0,080019908642,43.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,1000.0,958.61,...,0.0,0.0,"(0.0, 5, moda)","[31.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1000.0, 958.61, 883.97, 807.94, 729.47, 647.7...","[20.07, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[30, 31, 30, 31, 31, 30, 31]",0.00,-43.00,< -0.05
2488,8861781.0,073012143530,30.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2777.25,...,20.0,20.0,"(25.0, 2, moda)","[25.0, 25.0, 25.0, 31.0, 20.0, 20.0]","[3000.0, 2777.25, 2556.0, 2323.4, 2089.23, 184...","[19.74, 22.35, 16.82, 18.7, 16.82, 13.53, 12.9]","[30, 31, 30, 31, 31, 30, 31]",61.77,31.77,> 0.05
2496,23402939.0,080019991111,55.10,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1883.34,...,0.0,0.0,"(0.0, 5, moda)","[31.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2000.0, 1883.34, 1741.49, 1598.74, 1446.06, 1...","[14.68, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[30, 31, 30, 31, 31, 30, 31]",0.00,-55.10,< -0.05
2499,26269385.0,080019990651,27.10,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9655.59,...,15.0,13.0,"(13.0, 5, minimo)","[28.0, 34.0, 22.0, 31.0, 15.0, 13.0]","[10000.0, 9655.59, 9297.73, 8953.08, 8614.09, ...","[47.01, 74.92, 96.87, 105.0, 100.64, 90.43, 91...","[30, 31, 30, 31, 31, 30, 31]",25.86,-1.24,< -0.05


In [138]:
# Marcamos el metodo de calculo utilizado para conocer su efectivdad
dataset["metodo"] = dataset['fe_pago_frecuente'].str[2]

In [139]:
#Asi mismo ponemos en una columna aparte la fecha elegida para medir la efectividad
dataset["fecha_sel"] = dataset['fe_pago_frecuente'].str[0]

In [140]:
#la efectividad obtenida con el calculo minimo mejora
dataset[dataset["metodo"]=="minimo"]["group_dif_tea_frec"].value_counts(normalize = True)

+- 0.05    0.908065
> 0.05     0.079032
< -0.05    0.012903
Name: group_dif_tea_frec, dtype: float64

In [141]:
dataset[(dataset['group_dif_tea_frec']=='+- 0.05') & (dataset["metodo"]=='moda')]

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,fe_pago_frecuente,lista_pago_frecuente,lista_deuda,lista_dev,n_dias,tea_frecuente,dif_tea_frec,group_dif_tea_frec,metodo,fecha_sel
1,4215125.0,077012272390,26.82,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9534.57,...,"(10.0, 4, moda)","[10.0, 10.0, 10.0, 31.0, 10.0, -213278.0]","[10000.0, 9534.57, 9066.13, 8581.24, 8092.26, ...","[132.87, 133.07, 120.47, 119.76, 112.94, 100.8...","[30, 31, 30, 31, 31, 30, 31]",26.82,0.000000e+00,+- 0.05,moda,10.0
2,24088847.0,077012269280,40.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3744.43,...,"(15.0, 5, moda)","[34.0, 15.0, 15.0, 31.0, -5665.0, 15.0]","[4000.0, 3744.43, 3453.36, 3150.08, 2844.16, 2...","[83.1, 56.42, 48.76, 47.46, 40.16, 35.62, 33.04]","[30, 31, 30, 31, 31, 30, 31]",40.01,1.000000e-02,+- 0.05,moda,15.0
3,24182380.0,080019916301,32.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2785.29,...,"(15.0, 5, moda)","[47.0, 15.0, 15.0, 31.0, -12653.0, 15.0]","[3000.0, 2785.29, 2562.75, 2332.55, 2100.6, 18...","[39.59, 34.58, 29.82, 28.96, 24.44, 21.63, 20.01]","[30, 31, 30, 31, 31, 30, 31]",31.99,-1.000000e-02,+- 0.05,moda,15.0
4,22191616.0,080019919812,55.21,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1919.91,...,"(13.0, 5, moda)","[13.0, 22.0, 22.0, 31.0, 13.0, 13.0]","[2000.0, 1919.91, 1841.74, 1753.24, 1666.01, 1...","[41.95, 42.67, 36.34, 38.96, 37.03, 33.04, 32.87]","[30, 31, 30, 31, 31, 30, 31]",55.21,0.000000e+00,+- 0.05,moda,13.0
6,3131025.0,080019916311,43.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,5000.0,4672.09,...,"(18.0, 5, moda)","[34.0, 18.0, -57446.0, 31.0, 18.0, 18.0]","[5000.0, 4672.09, 4312.53, 3941.41, 3554.21, 3...","[85.17, 60.74, 51.72, 47.27, 46.2, 37.88, 35.71]","[30, 31, 30, 31, 31, 30, 31]",43.00,0.000000e+00,+- 0.05,moda,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,19407330.0,073012128630,42.58,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2500.0,2323.89,...,"(10.0, 4, moda)","[10.0, 10.0, 10.0, 31.0, 10.0, -9826.0]","[2500.0, 2323.89, 2144.77, 1957.71, 1766.93, 1...","[49.76, 48.59, 42.69, 40.93, 36.94, 31.25, 27.21]","[30, 31, 30, 31, 31, 30, 31]",42.58,0.000000e+00,+- 0.05,moda,10.0
2513,26207827.0,073012129200,45.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,1000.0,930.14,...,"(10.0, 4, moda)","[10.0, 10.0, 10.0, 31.0, 10.0, -12138.0]","[1000.0, 930.14, 859.04, 784.64, 708.69, 630.2...","[20.86, 20.38, 17.92, 17.19, 15.53, 13.14, 11.45]","[30, 31, 30, 31, 31, 30, 31]",44.98,-2.000000e-02,+- 0.05,moda,10.0
2514,24876802.0,071014209610,27.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2763.57,...,"(5.0, 5, moda)","[28.0, 5.0, 5.0, 31.0, 5.0, 5.0]","[3000.0, 2763.57, 2538.89, 2307.57, 2073.03, 1...","[36.07, 48.12, 42.49, 40.18, 36.1, 30.69, 27.64]","[30, 31, 30, 31, 31, 30, 31]",27.00,0.000000e+00,+- 0.05,moda,5.0
2515,24798402.0,073012127940,43.08,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3833.58,...,"(10.0, 4, moda)","[14571.0, 10.0, 10.0, 31.0, 10.0, 6933.0]","[4000.0, 3833.58, 3661.81, 3480.66, 3297.54, 3...","[84.47, 80.95, 73.61, 73.5, 69.63, 62.49, 58.56]","[30, 31, 30, 31, 31, 30, 31]",43.08,0.000000e+00,+- 0.05,moda,10.0


In [142]:
#Optamos por crear una función que mejor tome la fecha de pago mínima que sea mayor a 0
from scipy import stats as st
def minimo (fechas):
    fechas_truncate = [x for i,x in enumerate(fechas) if i!=3] #Excluimos de encontrar la moda al indice 3 que corresponde al calculo entre julio-agosto

    #Calculamos el valor mínimo mayor a 0
    fehasmayores0 = list(filter(lambda x: x > 0, fechas))
    valor_minimo = min(fehasmayores0,default=None)    

    indice_val_minimo = [i for i, n in enumerate(fechas) if n==valor_minimo][-1] if len(fehasmayores0)>0 else None
    return valor_minimo  #(fechas.index(valor_minimo) if len(fehasmayores0)>0 else None) #Por verificar
        #return None
    

In [143]:
#Agremamos la fecha de pago optima2 a partir de la fuincion de fecha de pago minima
dataset['fecha_min'] = dataset.apply(lambda x : minimo(x['lista_pago_frecuente']),axis=1)

In [144]:
#Solo para comparar agregamos una columna que indique si la fecha optima anterior es igual al a fecha optima2
dataset['fechasel_fechamin'] = dataset['fecha_min']==dataset['fecha_sel']

In [145]:
dataset.head()

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,lista_deuda,lista_dev,n_dias,tea_frecuente,dif_tea_frec,group_dif_tea_frec,metodo,fecha_sel,fecha_min,fechasel_fechamin
0,26260582.0,080019976972,30.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9747.14,...,"[10000.0, 9747.14, 9519.73, 9264.14, 9017.58, ...","[58.47, 78.45, 62.65, 74.57, 72.58, 64.11, 68.41]","[30, 31, 30, 31, 31, 30, 31]",30.00,0.00,+- 0.05,minimo,20.0,20.0,True
1,4215125.0,077012272390,26.82,Día Fijo Mensual,Cuota Fija,No es LA,1,8,10000.0,9534.57,...,"[10000.0, 9534.57, 9066.13, 8581.24, 8092.26, ...","[132.87, 133.07, 120.47, 119.76, 112.94, 100.8...","[30, 31, 30, 31, 31, 30, 31]",26.82,0.00,+- 0.05,moda,10.0,10.0,True
2,24088847.0,077012269280,40.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,4000.0,3744.43,...,"[4000.0, 3744.43, 3453.36, 3150.08, 2844.16, 2...","[83.1, 56.42, 48.76, 47.46, 40.16, 35.62, 33.04]","[30, 31, 30, 31, 31, 30, 31]",40.01,0.01,+- 0.05,moda,15.0,15.0,True
3,24182380.0,080019916301,32.00,Día Fijo Mensual,Cuota Fija,No es LA,1,8,3000.0,2785.29,...,"[3000.0, 2785.29, 2562.75, 2332.55, 2100.6, 18...","[39.59, 34.58, 29.82, 28.96, 24.44, 21.63, 20.01]","[30, 31, 30, 31, 31, 30, 31]",31.99,-0.01,+- 0.05,moda,15.0,15.0,True
4,22191616.0,080019919812,55.21,Día Fijo Mensual,Cuota Fija,No es LA,1,8,2000.0,1919.91,...,"[2000.0, 1919.91, 1841.74, 1753.24, 1666.01, 1...","[41.95, 42.67, 36.34, 38.96, 37.03, 33.04, 32.87]","[30, 31, 30, 31, 31, 30, 31]",55.21,0.00,+- 0.05,moda,13.0,13.0,True


In [146]:
#Verificamos el numero de fechas de pago frecuente con el metodo minimo es el mismo que el 2do metodo
dataset[(dataset['group_dif_tea_frec']=='+- 0.05') & (dataset["metodo"]=='minimo')]['fechasel_fechamin'].value_counts()

True    563
Name: fechasel_fechamin, dtype: int64

In [148]:
#Verificamos que casi todas las que fueron calculados con el metodo de moda da el mismo resultado que las del calculo con el metodo del minimo. a excepcion de 4 que serán revisadas
dataset[(dataset['group_dif_tea_frec']=='+- 0.05') & (dataset["metodo"]=='moda')]['fechasel_fechamin'].value_counts()

True     1706
False       4
Name: fechasel_fechamin, dtype: int64

In [149]:
dataset[(dataset['group_dif_tea_frec']=='+- 0.05') & (dataset["metodo"]=='moda')& (dataset["fechasel_fechamin"]==False)].iloc[:,24:]
#solo el cliente 75782 sale una tea incorrecta al considerar la fecha minima como la seleccionada
#Por lo tanto se utilizará la fecha de pago mínima calculada  de los 7 últimos meses para estimar la tea

,fpago_jun_jul,fpago_jul_ago,fpago_ago_sep,fpago_sep_oct,fe_pago_frecuente,lista_pago_frecuente,lista_deuda,lista_dev,n_dias,tea_frecuente,dif_tea_frec,group_dif_tea_frec,metodo,fecha_sel,fecha_min,fechasel_fechamin
192,14.0,31.0,14.0,14.0,"(14.0, 5, moda)","[13.0, 22.0, 14.0, 31.0, 14.0, 14.0]","[5000.0, 4113.96, 3798.71, 3453.15, 3096.04, 2...","[140.17, 122.22, 100.15, 96.81, 86.8, 71.71, 6...","[30, 31, 30, 31, 31, 30, 31]",79.60,0.01,+- 0.05,moda,14.0,13.0,False
945,14.0,31.0,13.0,13.0,"(14.0, 2, moda)","[14.0, 14.0, 14.0, 31.0, 13.0, 13.0]","[1870.0, 1814.15, 1805.79, 1783.45, 1778.41, 1...","[44.42, 45.82, 42.89, 45.04, 47.59, 44.65, 46.23]","[30, 31, 30, 31, 31, 30, 31]",69.58,-0.01,+- 0.05,moda,14.0,13.0,False
1068,30.0,31.0,0.0,30.0,"(30.0, 5, moda)","[16.0, 30.0, 30.0, 31.0, 0.0, 30.0]","[10000.0, 9366.43, 8092.05, 7446.06, 6798.9, 6...","[76.33, 76.45, 0.0, 1.95, 0.0, 0.0, 1.44]","[30, 31, 30, 31, 31, 30, 31]",9.92,0.02,+- 0.05,moda,30.0,16.0,False
2294,17.0,31.0,16.0,16.0,"(17.0, 2, moda)","[17.0, 17.0, 17.0, 31.0, 16.0, 16.0]","[736.0, 735.96, 735.91, 735.87, 735.82, 734.96...","[14.95, 16.11, 14.95, 16.11, 17.27, 16.09, 17.25]","[30, 31, 30, 31, 31, 30, 31]",74.52,0.00,+- 0.05,moda,17.0,16.0,False


In [156]:
dataset[dataset.index== 2294]

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,lista_deuda,lista_dev,n_dias,tea_frecuente,dif_tea_frec,group_dif_tea_frec,metodo,fecha_sel,fecha_min,fechasel_fechamin
2294,1650254.0,070011560211,74.52,Frecuencia Fija,Cuota Fija,Si es LA,1,8,736.0,735.96,...,"[736.0, 735.96, 735.91, 735.87, 735.82, 734.96...","[14.95, 16.11, 14.95, 16.11, 17.27, 16.09, 17.25]","[30, 31, 30, 31, 31, 30, 31]",74.52,0.0,+- 0.05,moda,17.0,16.0,False


In [157]:
#Optimizamos la función de calculo de fecha de pago. la cual devolverá la fecha de pago, el índice donde se encuentra y el metodo de calculo
from scipy import stats as st
def fpagominima (fechas):   
      
    #Calculamos el valor mínimo mayor a 0
    fehasmayores0 = list(filter(lambda x: x > 0, fechas))
    valor_minimo = min(fehasmayores0,default=0)    

    indice_val_minimo = [i for i, n in enumerate(fechas) if n==valor_minimo][-1] if len(fehasmayores0)>0 else 0 #último ínidce
    return valor_minimo , indice_val_minimo ,"minimo"
   
    

In [158]:
dataset['fec_sel_min_indice'] = dataset.apply(lambda x : fpagominima(x['lista_pago_frecuente']),axis=1)

In [159]:
dataset.iloc[:,29:41].head()
dataset.iloc[:,np.r_[0:1,28:41]].head()

,CODCLI,fe_pago_frecuente,lista_pago_frecuente,lista_deuda,lista_dev,n_dias,tea_frecuente,dif_tea_frec,group_dif_tea_frec,metodo,fecha_sel,fecha_min,fechasel_fechamin,fec_sel_min_indice
0,26260582.0,"(20.0, 5, minimo)","[27.0, 25.0, 25.0, 31.0, 20.0, 20.0]","[10000.0, 9747.14, 9519.73, 9264.14, 9017.58, ...","[58.47, 78.45, 62.65, 74.57, 72.58, 64.11, 68.41]","[30, 31, 30, 31, 31, 30, 31]",30.00,0.00,+- 0.05,minimo,20.0,20.0,True,"(20.0, 5, minimo)"
1,4215125.0,"(10.0, 4, moda)","[10.0, 10.0, 10.0, 31.0, 10.0, -213278.0]","[10000.0, 9534.57, 9066.13, 8581.24, 8092.26, ...","[132.87, 133.07, 120.47, 119.76, 112.94, 100.8...","[30, 31, 30, 31, 31, 30, 31]",26.82,0.00,+- 0.05,moda,10.0,10.0,True,"(10.0, 4, minimo)"
2,24088847.0,"(15.0, 5, moda)","[34.0, 15.0, 15.0, 31.0, -5665.0, 15.0]","[4000.0, 3744.43, 3453.36, 3150.08, 2844.16, 2...","[83.1, 56.42, 48.76, 47.46, 40.16, 35.62, 33.04]","[30, 31, 30, 31, 31, 30, 31]",40.01,0.01,+- 0.05,moda,15.0,15.0,True,"(15.0, 5, minimo)"
3,24182380.0,"(15.0, 5, moda)","[47.0, 15.0, 15.0, 31.0, -12653.0, 15.0]","[3000.0, 2785.29, 2562.75, 2332.55, 2100.6, 18...","[39.59, 34.58, 29.82, 28.96, 24.44, 21.63, 20.01]","[30, 31, 30, 31, 31, 30, 31]",31.99,-0.01,+- 0.05,moda,15.0,15.0,True,"(15.0, 5, minimo)"
4,22191616.0,"(13.0, 5, moda)","[13.0, 22.0, 22.0, 31.0, 13.0, 13.0]","[2000.0, 1919.91, 1841.74, 1753.24, 1666.01, 1...","[41.95, 42.67, 36.34, 38.96, 37.03, 33.04, 32.87]","[30, 31, 30, 31, 31, 30, 31]",55.21,0.00,+- 0.05,moda,13.0,13.0,True,"(13.0, 5, minimo)"


In [160]:
np.round((np.log(74.57/9264.14 + 1)*30 - np.log(62.65/9519.73 + 1)*31)/(np.log(74.57/9264.14 + 1)-np.log(62.65/9519.73 + 1)),0)

25.0

In [161]:
dataset[dataset.CODCLI==26260582.0].T

,0
CODCLI,26260582.0
PAGARE,080019976972
TASADE,30.0
MODALIDAD_PAGO,Día Fijo Mensual
TIPO_PLAN_PAGOS,Cuota Fija
LA,No es LA
PAGARES,1
MESES_CREDITO,8
DEUDA_ABR21,10000.0
DEUDA_MAY21,9747.14


In [162]:
dataset[dataset['fec_sel_min_indice'].str[0].isnull()]#.iloc[:,24:]

,CODCLI,PAGARE,TASADE,MODALIDAD_PAGO,TIPO_PLAN_PAGOS,LA,PAGARES,MESES_CREDITO,DEUDA_ABR21,DEUDA_MAY21,...,lista_dev,n_dias,tea_frecuente,dif_tea_frec,group_dif_tea_frec,metodo,fecha_sel,fecha_min,fechasel_fechamin,fec_sel_min_indice


In [163]:
#Calculamos la tea a partir de la última fecha de pago optima(minima)
dataset['tea_seleccionada'] = dataset.apply(lambda x : tea_mes_from_list(x['lista_deuda'],x['lista_dev'],x['fec_sel_min_indice'],x['n_dias']),axis=1)

In [164]:
dataset['dif_tea_sel'] = dataset['tea_seleccionada'] - dataset['TASADE']
bins_tea=[-np.inf,-0.049,0.05,np.inf]
labels_tea=['< -0.05', '+- 0.05','> 0.05']
dataset['group_dif_tea_sel'] = pd.cut(dataset['dif_tea_sel'], bins = bins_tea, labels = labels_tea)

In [165]:
dataset['group_dif_tea_sel'].value_counts(normalize=True)

+- 0.05    0.965363
> 0.05     0.024450
< -0.05    0.010187
Name: group_dif_tea_sel, dtype: float64

In [166]:
dataset.groupby(['TIPO_PLAN_PAGOS','LA','group_dif_tea_sel']).agg({'CODCLI':'count'})

CODCLI
TIPO_PLAN_PAGOS LA       group_dif_tea_sel        
Cuota Fija      No es LA < -0.05                21
                         +- 0.05              2313
                         > 0.05                 29
                Si es LA < -0.05                 4
                         +- 0.05                51
                         > 0.05                 29
Cuota Variable  No es LA < -0.05                 0
                         +- 0.05                 5
                         > 0.05                  2
                Si es LA < -0.05                 0
                         +- 0.05                 0
                         > 0.05                  0

In [169]:
dataset[(dataset['group_dif_tea_sel']!='+- 0.05')&(dataset['TIPO_PLAN_PAGOS']=='Cuota Fija')&(dataset['LA']=='No es LA')].iloc[:50,np.r_[0:1,28:44]]

,CODCLI,fe_pago_frecuente,lista_pago_frecuente,lista_deuda,lista_dev,n_dias,tea_frecuente,dif_tea_frec,group_dif_tea_frec,metodo,fecha_sel,fecha_min,fechasel_fechamin,fec_sel_min_indice,tea_seleccionada,dif_tea_sel,group_dif_tea_sel
39,26268168.0,"(0.0, 5, moda)","[31.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2000.0, 1868.77, 1716.09, 1561.36, 1403.17, 1...","[9.62, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[30, 31, 30, 31, 31, 30, 31]",0.00,-28.00,< -0.05,moda,0.0,31.0,False,"(31.0, 0, minimo)",NaN,NaN,NaN
62,22743384.0,"(28.0, 5, minimo)","[30.0, 29.0, 28.0, 31.0, 33.0, 28.0]","[6000.0, 5674.85, 5124.7, 4667.6, 4081.86, 359...","[5.37, 56.12, 27.58, 37.74, 18.3, 22.61, 27.89]","[30, 31, 30, 31, 31, 30, 31]",192.62,154.62,> 0.05,minimo,28.0,28.0,True,"(28.0, 5, minimo)",192.62,154.62,> 0.05
64,23702673.0,"(0.0, 5, moda)","[31.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2000.0, 1844.32, 1663.87, 1479.41, 1290.41, 1...","[6.21, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[30, 31, 30, 31, 31, 30, 31]",0.00,-45.08,< -0.05,moda,0.0,31.0,False,"(31.0, 0, minimo)",NaN,NaN,NaN
73,5054621.0,"(0.0, 5, moda)","[31.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2000.0, 1874.71, 1732.47, 1587.2, 1436.45, 12...","[4.67, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[30, 31, 30, 31, 31, 30, 31]",0.00,-52.10,< -0.05,moda,0.0,31.0,False,"(31.0, 0, minimo)",NaN,NaN,NaN
84,12856920.0,"(20.0, 4, minimo)","[27.0, 25.0, 25.0, 31.0, 20.0, 46.0]","[80000.0, 77111.35, 74335.49, 71405.68, 68504....","[294.79, 390.97, 308.23, 362.04, 347.33, 302.1...","[30, 31, 30, 31, 31, 30, 31]",18.00,3.50,> 0.05,minimo,20.0,20.0,True,"(20.0, 4, minimo)",18.00,3.50,> 0.05
173,28800.0,"(0.0, 5, moda)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2500.0, 2304.24, 2107.11, 1906.83, 1705.72, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[30, 31, 30, 31, 31, 30, 31]",0.00,-14.70,< -0.05,moda,0.0,NaN,False,"(0, 0, minimo)",0.00,-14.70,< -0.05
177,26242541.0,"(22.0, 4, minimo)","[50.0, 27.0, 23.0, 31.0, 22.0, 23.0]","[5700.0, 3760.63, 3552.07, 3315.35, 3073.57, 2...","[82.56, 51.71, 34.89, 37.08, 34.49, 28.04, 28.87]","[30, 31, 30, 31, 31, 30, 31]",56.05,-4.05,< -0.05,minimo,22.0,22.0,True,"(22.0, 4, minimo)",56.05,-4.05,< -0.05
189,12091192.0,"(0.0, 5, moda)","[31.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[4000.0, 3765.76, 3464.92, 3162.33, 2845.7, 25...","[50.33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[30, 31, 30, 31, 31, 30, 31]",0.00,-35.00,< -0.05,moda,0.0,31.0,False,"(31.0, 0, minimo)",NaN,NaN,NaN
224,24502724.0,"(0.0, 5, moda)","[31.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1000.0, 920.28, 830.3, 738.33, 644.09, 544.49...","[1.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[30, 31, 30, 31, 31, 30, 31]",0.00,-45.00,< -0.05,moda,0.0,31.0,False,"(31.0, 0, minimo)",NaN,NaN,NaN
236,24777669.0,"(14.0, 2, moda)","[14.0, 14.0, 14.0, 31.0, -4.0, -14.0]","[200000.0, 198867.56, 197783.84, 196631.85, 19...","[849.0, 897.07, 839.59, 886.99, 1840.37, 1777....","[30, 31, 30, 31, 31, 30, 31]",10.00,1.50,> 0.05,moda,14.0,14.0,True,"(14.0, 2, minimo)",10.00,1.50,> 0.05


In [177]:
dataset[dataset.index ==177].T

,177
CODCLI,26242541.0
PAGARE,080019956651
TASADE,60.1
MODALIDAD_PAGO,Día Fijo Mensual
TIPO_PLAN_PAGOS,Cuota Fija
LA,No es LA
PAGARES,1
MESES_CREDITO,8
DEUDA_ABR21,5700.0
DEUDA_MAY21,3760.63


In [232]:
np.round(np.power(35.51/1966.67 + 1, 360/(31-11))-1,4)*100

38.0

In [790]:
dataset.groupby(["metodo",'TIPO_PLAN_PAGOS','LA','group_dif_tea_frec']).agg({'CODCLI':'count'})

CODCLI
metodo TIPO_PLAN_PAGOS LA       group_dif_tea_frec        
minimo Cuota Fija      No es LA < -0.05                  8
                                +- 0.05                647
                                > 0.05                  21
                       Si es LA < -0.05                  5
                                +- 0.05                 58
                                > 0.05                  30
       Cuota Variable  No es LA < -0.05                  0
                                +- 0.05                  2
                                > 0.05                   1
                       Si es LA < -0.05                  0
                                +- 0.05                  0
                                > 0.05                   0
moda   Cuota Fija      No es LA < -0.05                 99
                                +- 0.05               2026
                                > 0.05                 112
                       Si es LA < -0.05                  0
                                +- 0.05                  7
                                > 0.05                   5
       Cuota Variable  No es LA < -0.05                  0
                                +- 0.05                  3
                                > 0.05                   1
                       Si es LA < -0.05                  0
                                +- 0.05                  0
                                > 0.05                   0

In [115]:
registoscaja = """
                select CODDEU,
       PERIODO,
       103 as CODEMP,
       REDESC as TIPCRE,
       MONEDA,
       sum(SALDO) SALDO,
       listagg(saldo,',')saldos,
       count(*) CREDITOS,
       listagg(decimal(TASADE,14,2),',') TEAS,
       decimal((sum(TASADE*saldo)/sum(saldo)),14,2)Ponderada,
       listagg(LA,',') LA
    from MODELOS.BASE_VIGENTES
    where PERIODO between 202101 and 202112 and coddeu <>0
    group by CODDEU,
             PERIODO,
             103,
             REDESC,
             MONEDA
      order by coddeu, periodo,tipcre,moneda
"""

In [116]:
#df_test = pd.read_sql_query(registoscaja,connection)
#df_test.head()

,CODDEU,PERIODO,CODEMP,TIPCRE,MONEDA,SALDO,SALDOS,CREDITOS,TEAS,PONDERADA,LA
0,40185.0,202101,103,9.0,1.0,5467.71,5467.71,1,34.50,34.5,NO
1,40185.0,202102,103,9.0,1.0,5467.71,5467.71,1,34.50,34.5,NO
2,40185.0,202103,103,9.0,1.0,5467.71,5467.71,1,34.50,34.5,NO
3,40185.0,202104,103,9.0,1.0,5467.71,5467.71,1,34.50,34.5,NO
4,40185.0,202105,103,9.0,1.0,5267.71,5267.71,1,34.50,34.5,NO


In [124]:
#df_test[df_test.TIPCRE==13].to_csv('TEAsTEST_CP_2021-REDESC13.csv',index=False)

In [27]:
def tea_mes (S,d,p,N):
    if p == N or S ==0:
        return None
    else:
        return np.round(np.power(d/S + 1, 360/(N-p))-1,4)*100
    

## Testeando con codigos deudor del rcc

In [80]:
query_test = """select D.CODDEU,
       D.AÑOREP * 10000 + D.MESREP * 100 + D.DIAREP PERIODO,
       D.CODEMP,
       D.TIPCRE,
       substr(CTACBE, 3, 1) MONEDA,
       --CTACBE,
       case
           when D.CTACBE like '14_1%' then 'Saldo'
           when D.CTACBE like '14_8%' then 'IntDev'
       end CTACBE,
       sum(SALDEU) SALDEU
    from SOFIAH.A5811 D
    where D.CODDEU in (154132414,135737844,194496796,16182478,127936200,188595413,123221893,16182478) -- 194496796,16182478,127936200,188595413,123221893,16182478,
          and AÑOREP = 2021
          and (CTACBE like '14_1%'
              or CTACBE like '14_8%') -- and codemp = 55 -- AND CODEMP = 228
    group by D.CODDEU,
             D.AÑOREP * 10000 + D.MESREP * 100 + D.DIAREP,
             D.CODEMP,
             D.TIPCRE,
             substr(CTACBE, 3, 1),
             --CTACBE,
             case
                 when D.CTACBE like '14_1%' then 'Saldo'
                 when D.CTACBE like '14_8%' then 'IntDev'
             end
             ORDER BY 1,2,3,4,5,6,7
             """

In [81]:
df_test = pd.read_sql_query(query_test,connection)
df_test

,CODDEU,PERIODO,CODEMP,TIPCRE,MONEDA,CTACBE,SALDEU
0,16182478.0,20210131.0,228.0,10,1,IntDev,98.89
1,16182478.0,20210131.0,228.0,10,1,Saldo,4405.00
2,16182478.0,20210228.0,228.0,10,1,IntDev,73.47
3,16182478.0,20210228.0,228.0,10,1,Saldo,3724.03
4,16182478.0,20210331.0,228.0,10,1,IntDev,67.65
...,...,...,...,...,...,...,...
329,194496796.0,20211031.0,6.0,13,1,Saldo,41364.49
330,194496796.0,20211130.0,6.0,13,1,IntDev,451.04
331,194496796.0,20211130.0,6.0,13,1,Saldo,41307.35
332,194496796.0,20211231.0,6.0,13,1,IntDev,27.99


In [82]:
df_test.query('PERIODO == 20210228 & CODEMP == 73'  ).values

array([[135737844.0, 20210228.0, 73.0, '11', '1', 'IntDev', 15.29],
       [135737844.0, 20210228.0, 73.0, '11', '1', 'Saldo', 612.66],
       [154132414.0, 20210228.0, 73.0, '11', '1', 'IntDev', 4.15],
       [154132414.0, 20210228.0, 73.0, '11', '1', 'Saldo', 412.38]],
      dtype=object)

In [83]:
df_test_pivot = df_test.pivot(index=['CODDEU','CODEMP','TIPCRE','MONEDA'],columns = ['PERIODO','CTACBE'],values = 'SALDEU')

In [84]:
#Configuraciones adicionales
#Mostrar todas las columnas de pandas
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 150)

In [85]:
df_test_pivot = df_test_pivot.fillna(0)
df_test_pivot

PERIODO                          20210131.0            20210228.0             \
CTACBE                               IntDev      Saldo     IntDev      Saldo   
CODDEU      CODEMP TIPCRE MONEDA                                               
16182478.0  228.0  10     1           98.89    4405.00      73.47    3724.03   
123221893.0 2.0    11     1            0.15       0.00       0.00       0.00   
                          2            0.00       0.00       0.00       0.00   
            6.0    11     2            0.00       0.00       0.15       0.00   
            82.0   11     1            0.00     129.90       0.00      44.90   
            110.0  12     1            0.00       0.00       0.00       0.00   
            142.0  11     1            0.00      35.59       0.00       0.00   
                   12     1            0.00       0.00       0.00       0.00   
127936200.0 4.0    12     1            8.00   32381.89       0.00       0.00   
            6.0    11     1            0.00       0.00       0.00       0.00   
            123.0  13     1         2098.06  158102.14    1596.33  157551.18   
            142.0  11     1            0.00     342.64       0.00       0.00   
135737844.0 73.0   11     1            0.00     161.19      15.29     612.66   
            103.0  12     1            5.47    6285.90       0.00    6169.64   
            145.0  11     1            0.00      79.00       0.00      48.20   
154132414.0 6.0    11     1            2.74    1514.61       0.37    1274.49   
                          2            0.18       8.26       0.00       0.00   
            73.0   11     1            7.49     626.09       4.15     412.38   
            103.0  12     1           36.38   25132.11      12.84   24402.06   
            142.0  11     1            0.00    2818.95       0.00    1211.67   
                   12     1            0.00       0.00       0.00       0.00   
188595413.0 103.0  10     1           52.19    1753.37      42.21    1605.89   
            104.0  10     1           35.51    1966.67      26.75    1745.51   
194496796.0 6.0    13     1          470.61   41800.01     427.35   41761.67   
            103.0  12     1            1.54    1775.62       0.00    1486.65   

PERIODO                          20210331.0           20210430.0            \
CTACBE                               IntDev     Saldo     IntDev     Saldo   
CODDEU      CODEMP TIPCRE MONEDA                                             
16182478.0  228.0  10     1           67.65   3013.40      49.39   2292.62   
123221893.0 2.0    11     1           14.60   2388.68       0.00      0.00   
                          2            0.00      0.00       0.00      0.00   
            6.0    11     2            0.00      0.00       0.00      0.00   
            82.0   11     1           92.36   4044.90      78.05   3756.39   
            110.0  12     1            0.00      0.00       0.00      0.00   
            142.0  11     1            0.00      0.00       0.00      0.00   
                   12     1            1.78   2000.00       0.55   1843.08   
127936200.0 4.0    12     1            0.00      0.00       0.00      0.00   
            6.0    11     1            0.00      0.00       0.00      0.00   
            123.0  13     1            0.00      0.00       0.00      0.00   
            142.0  11     1            0.00    168.32       0.00    441.13   
135737844.0 73.0   11     1            0.78    389.47       0.00      0.00   
            103.0  12     1            5.26   6046.92       3.44   5928.51   
            145.0  11     1            0.00    320.00       0.00      0.00   
154132414.0 6.0    11     1            0.00   1077.05       0.00     10.56   
                          2            0.00      0.00       0.23     34.08   
            73.0   11     1            2.81    209.01       0.00      0.00   
            103.0  12     1           32.55  23642.39      27.37  22894.32   
            142.0  11     1            0.00

In [86]:
tea_mes(3559.14,74,7,30)

38.0

In [87]:
#Estructuramos la tabla de tal forma que todas las columnas sean solamente los saldos y los intereses devengados de todos los periodos
level1 = df_test_pivot.columns.get_level_values(0).astype(int).astype(str)
level2 = df_test_pivot.columns.get_level_values(1)
df_test_pivot.columns = level2+"_"+level1

In [88]:
df_test_pivot

IntDev_20210131  Saldo_20210131  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 98.89         4405.00   
123221893.0 2.0    11     1                  0.15            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                  0.00          129.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00           35.59   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  8.00        32381.89   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               2098.06       158102.14   
            142.0  11     1                  0.00          342.64   
135737844.0 73.0   11     1                  0.00          161.19   
            103.0  12     1                  5.47         6285.90   
            145.0  11     1                  0.00           79.00   
154132414.0 6.0    11     1                  2.74         1514.61   
                          2                  0.18            8.26   
            73.0   11     1                  7.49          626.09   
            103.0  12     1                 36.38        25132.11   
            142.0  11     1                  0.00         2818.95   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 52.19         1753.37   
            104.0  10     1                 35.51         1966.67   
194496796.0 6.0    13     1                470.61        41800.01   
            103.0  12     1                  1.54         1775.62   

                                  IntDev_20210228  Saldo_20210228  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 73.47         3724.03   
123221893.0 2.0    11     1                  0.00            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.15            0.00   
            82.0   11     1                  0.00           44.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00            0.00   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  0.00            0.00   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               1596.33       157551.18   
            142.0  11     1                  0.00            0.00   
135737844.0 73.0   11     1                 15.29          612.66   
            103.0  12     1                  0.00         6169.64   
            145.0  11     1                  0.00           48.20   
154132414.0 6.0    11     1                  0.37         1274.49   
                          2                  0.00            0.00   
            73.0   11     1                  4.15          412.38   
            103.0  12     1                 12.84        24402.06   
            142.0  11     1                  0.00         1211.67   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 42.21         1605.89   
            104.0  10     1                 26.75         1745.51   
194496796.0 6.0    13     1                427.35        41761.67   
            103.0  12     1                  0.00         1486.65   

                                  IntDev_20210331  Saldo_20210331  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 67.65         3013.40   
123221893.0 2.0    11     1                 14.60         2388.68   
                          2   

In [89]:
name_columns = df_test_pivot.columns
days_month_columns = [int(x[-2:]) for x in name_columns]
days_month_columns

[31,
 31,
 28,
 28,
 31,
 31,
 30,
 30,
 31,
 31,
 30,
 30,
 31,
 31,
 31,
 31,
 30,
 30,
 31,
 31,
 30,
 30,
 31,
 31]

In [90]:
print(days_month_columns[0::2])


[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


In [91]:
df_test_pivot['lista_saldos'] = df_test_pivot.filter(like='Saldo_', axis=1).values.tolist()
df_test_pivot['lista_interes_dev'] = df_test_pivot.filter(like='IntDev_', axis=1).values.tolist()
df_test_pivot['n_dias'] = [days_month_columns[0::2] for _ in range(len(df_test_pivot))]

In [92]:
df_test_pivot

IntDev_20210131  Saldo_20210131  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 98.89         4405.00   
123221893.0 2.0    11     1                  0.15            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                  0.00          129.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00           35.59   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  8.00        32381.89   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               2098.06       158102.14   
            142.0  11     1                  0.00          342.64   
135737844.0 73.0   11     1                  0.00          161.19   
            103.0  12     1                  5.47         6285.90   
            145.0  11     1                  0.00           79.00   
154132414.0 6.0    11     1                  2.74         1514.61   
                          2                  0.18            8.26   
            73.0   11     1                  7.49          626.09   
            103.0  12     1                 36.38        25132.11   
            142.0  11     1                  0.00         2818.95   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 52.19         1753.37   
            104.0  10     1                 35.51         1966.67   
194496796.0 6.0    13     1                470.61        41800.01   
            103.0  12     1                  1.54         1775.62   

                                  IntDev_20210228  Saldo_20210228  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 73.47         3724.03   
123221893.0 2.0    11     1                  0.00            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.15            0.00   
            82.0   11     1                  0.00           44.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00            0.00   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  0.00            0.00   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               1596.33       157551.18   
            142.0  11     1                  0.00            0.00   
135737844.0 73.0   11     1                 15.29          612.66   
            103.0  12     1                  0.00         6169.64   
            145.0  11     1                  0.00           48.20   
154132414.0 6.0    11     1                  0.37         1274.49   
                          2                  0.00            0.00   
            73.0   11     1                  4.15          412.38   
            103.0  12     1                 12.84        24402.06   
            142.0  11     1                  0.00         1211.67   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 42.21         1605.89   
            104.0  10     1                 26.75         1745.51   
194496796.0 6.0    13     1                427.35        41761.67   
            103.0  12     1                  0.00         1486.65   

                                  IntDev_20210331  Saldo_20210331  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 67.65         3013.40   
123221893.0 2.0    11     1                 14.60         2388.68   
                          2   

In [93]:
def agruparSaldosDescendentes(lista):
    listaDescensos = []
    listaDescensos.append([])
    listaDescensos[0].append(lista[0])
    numlist, first = 0,0
    
    for j in range(first+1, len(lista)):
        if lista[first] >= lista[j]:
            listaDescensos[numlist].append(lista[j])
        else:
            numlist += 1
            listaDescensos.append([])
            listaDescensos[numlist].append(lista[j])
        first+=1    
    return listaDescensos
#agruparSaldosDescendentes(liston)

In [94]:
df_test_pivot['lista_saldos_desc'] = df_test_pivot["lista_saldos"].apply(agruparSaldosDescendentes)


In [95]:
df_test_pivot

IntDev_20210131  Saldo_20210131  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 98.89         4405.00   
123221893.0 2.0    11     1                  0.15            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                  0.00          129.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00           35.59   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  8.00        32381.89   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               2098.06       158102.14   
            142.0  11     1                  0.00          342.64   
135737844.0 73.0   11     1                  0.00          161.19   
            103.0  12     1                  5.47         6285.90   
            145.0  11     1                  0.00           79.00   
154132414.0 6.0    11     1                  2.74         1514.61   
                          2                  0.18            8.26   
            73.0   11     1                  7.49          626.09   
            103.0  12     1                 36.38        25132.11   
            142.0  11     1                  0.00         2818.95   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 52.19         1753.37   
            104.0  10     1                 35.51         1966.67   
194496796.0 6.0    13     1                470.61        41800.01   
            103.0  12     1                  1.54         1775.62   

                                  IntDev_20210228  Saldo_20210228  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 73.47         3724.03   
123221893.0 2.0    11     1                  0.00            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.15            0.00   
            82.0   11     1                  0.00           44.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00            0.00   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  0.00            0.00   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               1596.33       157551.18   
            142.0  11     1                  0.00            0.00   
135737844.0 73.0   11     1                 15.29          612.66   
            103.0  12     1                  0.00         6169.64   
            145.0  11     1                  0.00           48.20   
154132414.0 6.0    11     1                  0.37         1274.49   
                          2                  0.00            0.00   
            73.0   11     1                  4.15          412.38   
            103.0  12     1                 12.84        24402.06   
            142.0  11     1                  0.00         1211.67   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 42.21         1605.89   
            104.0  10     1                 26.75         1745.51   
194496796.0 6.0    13     1                427.35        41761.67   
            103.0  12     1                  0.00         1486.65   

                                  IntDev_20210331  Saldo_20210331  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 67.65         3013.40   
123221893.0 2.0    11     1                 14.60         2388.68   
                          2   

In [96]:
#Función para reagrupar el resto de listas en función a la de saldos descendientes
def reshape(listaAgrupada, listaPorAgrupar):
    last = 0
    res = []
    for ele in listaAgrupada:
        res.append(listaPorAgrupar[last : last + len(ele)])
        last += len(ele)
          
    return res

In [97]:
#df_test_pivot['lista_saldos'] = df_test_pivot.apply(lambda x : reshape(x['lista_saldos_desc'],x['lista_saldos']),axis=1)
df_test_pivot['lista_interes_dev'] = df_test_pivot.apply(lambda x : reshape(x['lista_saldos_desc'],x['lista_interes_dev']),axis=1)
df_test_pivot['n_dias'] = df_test_pivot.apply(lambda x : reshape(x['lista_saldos_desc'],x['n_dias']),axis=1)

In [98]:
df_test_pivot

IntDev_20210131  Saldo_20210131  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 98.89         4405.00   
123221893.0 2.0    11     1                  0.15            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                  0.00          129.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00           35.59   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  8.00        32381.89   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               2098.06       158102.14   
            142.0  11     1                  0.00          342.64   
135737844.0 73.0   11     1                  0.00          161.19   
            103.0  12     1                  5.47         6285.90   
            145.0  11     1                  0.00           79.00   
154132414.0 6.0    11     1                  2.74         1514.61   
                          2                  0.18            8.26   
            73.0   11     1                  7.49          626.09   
            103.0  12     1                 36.38        25132.11   
            142.0  11     1                  0.00         2818.95   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 52.19         1753.37   
            104.0  10     1                 35.51         1966.67   
194496796.0 6.0    13     1                470.61        41800.01   
            103.0  12     1                  1.54         1775.62   

                                  IntDev_20210228  Saldo_20210228  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 73.47         3724.03   
123221893.0 2.0    11     1                  0.00            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.15            0.00   
            82.0   11     1                  0.00           44.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00            0.00   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  0.00            0.00   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               1596.33       157551.18   
            142.0  11     1                  0.00            0.00   
135737844.0 73.0   11     1                 15.29          612.66   
            103.0  12     1                  0.00         6169.64   
            145.0  11     1                  0.00           48.20   
154132414.0 6.0    11     1                  0.37         1274.49   
                          2                  0.00            0.00   
            73.0   11     1                  4.15          412.38   
            103.0  12     1                 12.84        24402.06   
            142.0  11     1                  0.00         1211.67   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 42.21         1605.89   
            104.0  10     1                 26.75         1745.51   
194496796.0 6.0    13     1                427.35        41761.67   
            103.0  12     1                  0.00         1486.65   

                                  IntDev_20210331  Saldo_20210331  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 67.65         3013.40   
123221893.0 2.0    11     1                 14.60         2388.68   
                          2   

In [99]:
#Función para encontrar el día de pago de cuota iterando entre listas 

def calculateFPago_iterador(list_saldo,list_dev,list_n,iterar_todos=True):
    ciclos_decrecientes = len(list_saldo)
    result = []
    #print("Total ciclos: ",ciclos_decrecientes)
    for i in range(ciclos_decrecientes):
        result.append([])
        #print("Comenzando ciclo ",i," en ", list_saldo)
        saldos = list_saldo[i]
        devengados = list_dev[i]
        dias = list_n[i]
        #print("============ciclo=============",i)
        for index,n in enumerate(dias):
            for index_,n_ in enumerate(dias):
                condicion_opcional = True if iterar_todos else (index+1 == index_)
                if((index <index_) & condicion_opcional) : #  & (n != n_)
                    #print (f"compare {index} con {index_}, fechas {n} y {n_}")
                    _fpago= get_fpago(saldos[index],devengados[index],n,saldos[index_],devengados[index_],n_)
                    #print(_fpago)       
                    result[i].append(_fpago)
                    #print(result)
    return result
            
          

In [100]:
def calculaTEAs_ciclos(saldo_list,dev_list,dias_list,fpago_between_list, metodo='left'):
    ciclos_decrecientes = len(saldo_list)
    result = []
    
    for i in range(ciclos_decrecientes):
        result.append([])
        
        #Listas auxiliares
        list_left = fpago_between_list[i] + [None]
        list_right = [None] + fpago_between_list[i]

        #Iterar
        if(metodo == 'left'):        
           
            for index,elemento in enumerate(list_left):
                if(elemento == None):
                    result[i].append("ND")
                else:
                    #print("Parametros: ",saldo_list[i][index],dev_list[i][index],elemento,dias_list[i][index])
                    tea_calc = tea_mes(saldo_list[i][index],dev_list[i][index],elemento,dias_list[i][index])
                    #print("Resultado: ",tea_calc)
                    result[i].append(tea_calc)
            

        elif(metodo == 'right'):
            
            for index,elemento in enumerate(list_right):
                if(elemento == None):
                    result[i].append("ND")
                else:
                    tea_calc = tea_mes(saldo_list[i][index],dev_list[i][index],elemento,dias_list[i][index])
                    result[i].append(tea_calc)
            

        elif(metodo == 'both'):
            for index,elemento in enumerate(list_left):
                if(elemento == None):
                    result[i].append("ND")
                else:
                    tea_calc = tea_mes(saldo_list[i][index],dev_list[i][index],elemento,dias_list[i][index])
                    result[i].append(tea_calc)
            for index,elemento in enumerate(list_right):
                if(elemento == None):
                    result[i].append("ND")
                else:
                    tea_calc = tea_mes(saldo_list[i][index],dev_list[i][index],elemento,dias_list[i][index])
                    result[i].append(tea_calc)
            
    return result
                                                                                                                                                                                                                                                                                                                                                        
    
    

In [101]:
#Pruebas de formulas

In [102]:
saldo_list =[[1966.67, 1745.51, 1513.1], [4000.0, 3559.14, 3559.14, 3327.63], [3595.72, 3305.59, 3009.96, 2704.05, 2391.81]]
dev_list = [[35.51, 26.75, 27.32], [83.16, 0.0, 74.0, 72.22], [72.75, 66.61, 60.94, 54.54, 48.48]]
dias_list =[[31, 28, 31], [30, 31, 30, 31], [31, 30, 31, 30, 31]]
ciclos_decrecientes = len(saldo_list)
print (ciclos_decrecientes)

3


In [103]:
saldo_list =[[41800.01, 41761.67, 41640.52, 41600.49, 41545.77, 41504.74, 41449.04, 41406.99, 41364.49, 41364.49, 41307.35, 41205.63]]
dev_list = [[470.61, 427.35, 28.28, 14.13, 28.22, 14.09, 28.15, 28.12, 14.05, 451.5, 451.04, 27.99]]
dias_list =[[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]]
ciclos_decrecientes = len(saldo_list)
print (ciclos_decrecientes)

1


In [104]:
dev_list =[[98.89, 73.47, 67.65, 49.39, 34.78, 16.26], [174.6, 141.7, 93.08, 101.67, 91.36, 94.32]]
dias_list =[[31, 28, 31, 30, 31, 30], [31, 31, 30, 31, 30, 31]]
saldo_list= [[4405.0, 3724.03, 3013.4, 2292.62, 1549.12, 787.85], [30000.0, 27835.61, 25618.23, 23308.55, 20945.75, 18528.61]]
ciclos_decrecientes = len(saldo_list)
print (ciclos_decrecientes)

2


In [105]:
dev_list =[[0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0]]
dias_list =[[31, 28], [31, 30, 31], [30, 31, 31, 30, 31], [30, 31]]
saldo_list=[[0.0, 0.0], [2388.68, 0.0, 0.0], [254.92, 168.09, 168.09, 15.49, 15.49], [400.1, 8.5]]
ciclos_decrecientes = len(saldo_list)
print (ciclos_decrecientes)

4


In [106]:
dev_list = [[0.0, 0.0, 92.36, 78.05, 69.35, 65.13, 0.0, 0.0, 43.81, 0.0, 0.0, 0.0]]
dias_list = [[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]]
saldo_list= [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
ciclos_decrecientes = len(saldo_list)
print (ciclos_decrecientes)

1


In [107]:
fpago_between_list = calculateFPago_iterador(saldo_list,dev_list,dias_list,False)
fpago_between_list

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [108]:
resultado = calculaTEAs_ciclos(saldo_list,dev_list,dias_list,fpago_between_list, metodo='left')
resultado

[[None, None, None, None, None, None, None, None, None, None, None, 'ND']]

In [109]:
resultado2 = calculaTEAs_ciclos(saldo_list,dev_list,dias_list,fpago_between_list, metodo='left')
resultado2

[[None, None, None, None, None, None, None, None, None, None, None, 'ND']]

In [110]:
#Ejecución en tabla

In [111]:
df_test_pivot["f_pago_inter"] = df_test_pivot.apply(lambda x : calculateFPago_iterador(x['lista_saldos_desc'],x['lista_interes_dev'],x['n_dias'],False),axis=1)

<ipython-input-35-334793f7408d>:14: RuntimeWarning: invalid value encountered in double_scalars
  return np.round((np.log(d2/s2 + 1)*N1 - np.log(d1/s1 + 1)*N2)/(np.log(d2/s2 + 1)-np.log(d1/s1 + 1)),0)


In [112]:
df_test_pivot.head()

IntDev_20210131  Saldo_20210131  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 98.89          4405.0   
123221893.0 2.0    11     1                  0.15             0.0   
                          2                  0.00             0.0   
            6.0    11     2                  0.00             0.0   
            82.0   11     1                  0.00           129.9   

                                  IntDev_20210228  Saldo_20210228  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 73.47         3724.03   
123221893.0 2.0    11     1                  0.00            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.15            0.00   
            82.0   11     1                  0.00           44.90   

                                  IntDev_20210331  Saldo_20210331  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 67.65         3013.40   
123221893.0 2.0    11     1                 14.60         2388.68   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                 92.36         4044.90   

                                  IntDev_20210430  Saldo_20210430  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 49.39         2292.62   
123221893.0 2.0    11     1                  0.00            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                 78.05         3756.39   

                                  IntDev_20210531  Saldo_20210531  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 34.78         1549.12   
123221893.0 2.0    11     1                  0.00            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                 69.35         3453.57   

                                  IntDev_20210630  Saldo_20210630  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 16.26          787.85   
123221893.0 2.0    11     1                 36.30          254.92   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                 65.13         3142.05   

                                  IntDev_20210731  Saldo_20210731  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                174.60        30000.00   
123221893.0 2.0    11     1                  0.00          168.09   
                          2                  1.21          149.48   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                  0.00         2457.13   

                                  IntDev_20210831  Saldo_20210831  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                141.70        27835.61   
123221893.0 2.0    11     1                  0.00          168.09   
                          2                  1.22          151.10   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                  0.00         2083.03   

                                  IntDev_20210930  Saldo_20210930  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10   

In [113]:
df_test_pivot["tea_calculada"] = df_test_pivot.apply(lambda x : calculaTEAs_ciclos(x['lista_saldos_desc'],x['lista_interes_dev'],x['n_dias'],x['f_pago_inter'],'left'),axis=1)

In [114]:
df_test_pivot

IntDev_20210131  Saldo_20210131  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 98.89         4405.00   
123221893.0 2.0    11     1                  0.15            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.00            0.00   
            82.0   11     1                  0.00          129.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00           35.59   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  8.00        32381.89   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               2098.06       158102.14   
            142.0  11     1                  0.00          342.64   
135737844.0 73.0   11     1                  0.00          161.19   
            103.0  12     1                  5.47         6285.90   
            145.0  11     1                  0.00           79.00   
154132414.0 6.0    11     1                  2.74         1514.61   
                          2                  0.18            8.26   
            73.0   11     1                  7.49          626.09   
            103.0  12     1                 36.38        25132.11   
            142.0  11     1                  0.00         2818.95   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 52.19         1753.37   
            104.0  10     1                 35.51         1966.67   
194496796.0 6.0    13     1                470.61        41800.01   
            103.0  12     1                  1.54         1775.62   

                                  IntDev_20210228  Saldo_20210228  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 73.47         3724.03   
123221893.0 2.0    11     1                  0.00            0.00   
                          2                  0.00            0.00   
            6.0    11     2                  0.15            0.00   
            82.0   11     1                  0.00           44.90   
            110.0  12     1                  0.00            0.00   
            142.0  11     1                  0.00            0.00   
                   12     1                  0.00            0.00   
127936200.0 4.0    12     1                  0.00            0.00   
            6.0    11     1                  0.00            0.00   
            123.0  13     1               1596.33       157551.18   
            142.0  11     1                  0.00            0.00   
135737844.0 73.0   11     1                 15.29          612.66   
            103.0  12     1                  0.00         6169.64   
            145.0  11     1                  0.00           48.20   
154132414.0 6.0    11     1                  0.37         1274.49   
                          2                  0.00            0.00   
            73.0   11     1                  4.15          412.38   
            103.0  12     1                 12.84        24402.06   
            142.0  11     1                  0.00         1211.67   
                   12     1                  0.00            0.00   
188595413.0 103.0  10     1                 42.21         1605.89   
            104.0  10     1                 26.75         1745.51   
194496796.0 6.0    13     1                427.35        41761.67   
            103.0  12     1                  0.00         1486.65   

                                  IntDev_20210331  Saldo_20210331  \
CODDEU      CODEMP TIPCRE MONEDA                                    
16182478.0  228.0  10     1                 67.65         3013.40   
123221893.0 2.0    11     1                 14.60         2388.68   
                          2   

In [199]:
def calcular_fpago (nombre, data):
    
    return value[1]+name

In [204]:
days_month_columns

[31,
 31,
 28,
 28,
 31,
 31,
 30,
 30,
 31,
 31,
 30,
 30,
 31,
 31,
 31,
 31,
 30,
 30,
 31,
 31,
 30,
 30]

In [222]:
def prueba(columna,listado):
    return listado + columna[1]

In [223]:
df_test_pivot['prueba2'] = df_test_pivot.apply(lambda x : prueba(x,days_month_columns[0]),axis=1)

df_test_pivot['prueba2']

In [224]:
df_test_pivot

Saldo_20210131  IntDev_20210131  Saldo_20210228  \
CODDEU      CODEMP TIPCRE                                                    
16182478.0  228.0  10             4405.00            98.89         3724.03   
127936200.0 4.0    12            32381.89             8.00             NaN   
            6.0    11                 NaN              NaN             NaN   
            123.0  13           158102.14          2098.06       157551.18   
            142.0  11              342.64              NaN             NaN   

                           IntDev_20210228  Saldo_20210331  IntDev_20210331  \
CODDEU      CODEMP TIPCRE                                                     
16182478.0  228.0  10                73.47         3013.40            67.65   
127936200.0 4.0    12                  NaN             NaN              NaN   
            6.0    11                  NaN             NaN              NaN   
            123.0  13              1596.33             NaN              NaN   
            142.0  11                  NaN          168.32              NaN   

                           Saldo_20210430  IntDev_20210430  Saldo_20210531  \
CODDEU      CODEMP TIPCRE                                                    
16182478.0  228.0  10             2292.62            49.39         1549.12   
127936200.0 4.0    12                 NaN              NaN             NaN   
            6.0    11                 NaN              NaN             NaN   
            123.0  13                 NaN              NaN             NaN   
            142.0  11              441.13              NaN          506.47   

                           IntDev_20210531  ...  IntDev_20210731  \
CODDEU      CODEMP TIPCRE                   ...                    
16182478.0  228.0  10                34.78  ...           174.60   
127936200.0 4.0    12                  NaN  ...              NaN   
            6.0    11                  NaN  ...            34.67   
            123.0  13                  NaN  ...              NaN   
            142.0  11                  NaN  ...            36.72   

                           Saldo_20210831  IntDev_20210831  Saldo_20210930  \
CODDEU      CODEMP TIPCRE                                                    
16182478.0  228.0  10            27835.61            141.7        25618.23   
127936200.0 4.0    12                 NaN              NaN             NaN   
            6.0    11                 NaN              NaN             NaN   
            123.0  13                 NaN              NaN             NaN   
            142.0  11              334.23              NaN          769.94   

                           IntDev_20210930  Saldo_20211031  IntDev_20211031  \
CODDEU      CODEMP TIPCRE                                                     
16182478.0  228.0  10                93.08        23308.55           101.67   
127936200.0 4.0    12                  NaN             NaN              NaN   
            6.0    11                  NaN             NaN              NaN   
            123.0  13                  NaN             NaN              NaN   
            142.0  11                  NaN          974.40              NaN   

                           Saldo_20211130  IntDev_20211130  prueba2  
CODDEU      CODEMP TIPCRE                                            
16182478.0  228.0  10            20945.75            91.36   129.89  
127936200.0 4.0    12                 NaN              NaN    39.00  
            6.0    11                 NaN              NaN      NaN  
            123.0  13                 NaN              NaN  2129.06  
            142.0  11              464.21              NaN      NaN  

[5 rows x 23 columns]

In [227]:
list(filter(lambda c: 'Saldo_' in c, df_test_pivot.columns.tolist()))

['Saldo_20210131',
 'Saldo_20210228',
 'Saldo_20210331',
 'Saldo_20210430',
 'Saldo_20210531',
 'Saldo_20210630',
 'Saldo_20210731',
 'Saldo_20210831',
 'Saldo_20210930',
 'Saldo_20211031',
 'Saldo_20211130']

In [371]:
df_test_pivot.drop("prueba",axis =1,inplace=True)

In [226]:
df_test_pivot.iloc[:,0:1]

Saldo_20210131
CODDEU      CODEMP TIPCRE                
16182478.0  228.0  10             4405.00
127936200.0 4.0    12            32381.89
            6.0    11                 NaN
            123.0  13           158102.14
            142.0  11              342.64

In [225]:
list(filter(lambda c: 'fpago' in c, df_test_pivot.columns.tolist()))

[]

## Probando con pycaret

In [48]:
datatemp.head()


,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


In [11]:
data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions ' + str(data_unseen.shape))

Data for Modeling: (3543, 15)
Unseen Data For Predictions (394, 15)


In [12]:
from pycaret.regression import *


In [15]:
exp_reg102 = setup(data = data, target = 'TASADE', session_id=123,
                  normalize = True, transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95, 
                  bin_numeric_features = ['DEUDA_ENE21', 'DEUDA_FEB21', 'DEUDA_MAR21', 'DEUDA_ABR21',
       'DEUDA_MAY21', 'DEV_ENE21', 'DEV_FEB21', 'DEV_MAR21', 'DEV_ABR21',
       'DEV_MAY21', 'var_saldo_ene_feb', 'var_saldo_feb_mar',
       'var_saldo_mar_abr', 'var_saldo_abr_may'],
                  log_experiment = True, experiment_name = 'TASADE1')

,Description,Value
0,session_id,123
1,Target,TASADE
2,Original Data,"(3543, 15)"
3,Missing Values,False
4,Numeric Features,14
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(2480, 156)"


2021/07/17 10:50:33 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



In [16]:
top3 = compare_models(exclude = ['ransac'], n_select = 3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,6.1647,72.7325,8.5110,0.6367,0.2185,0.1694,0.1510
catboost,CatBoost Regressor,6.0961,72.9330,8.5185,0.6367,0.2236,0.1711,5.0740
rf,Random Forest Regressor,6.1364,75.4058,8.6611,0.6242,0.2236,0.1705,0.6160
xgboost,Extreme Gradient Boosting,6.2569,77.7541,8.8048,0.6116,0.2282,0.1734,0.4200
gbr,Gradient Boosting Regressor,6.5170,82.4714,9.0635,0.5884,0.2398,0.1891,0.1960
br,Bayesian Ridge,6.5032,82.1017,9.0497,0.5882,0.2403,0.1858,0.0520
omp,Orthogonal Matching Pursuit,6.8126,89.8962,9.4701,0.5489,0.2490,0.1969,0.0190
knn,K Neighbors Regressor,6.8044,91.3164,9.5280,0.5448,0.2487,0.1930,0.7150
et,Extra Trees Regressor,6.8879,100.1798,9.9891,0.4994,0.2589,0.1879,0.6920
dt,Decision Tree Regressor,7.0224,106.1888,10.2805,0.4694,0.2697,0.1915,0.0270


In [17]:

type(top3)

list

In [18]:
print(top3)


[PowerTransformedTargetRegressor(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_leaves=31, objective=None,
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                random_state=1...
                                                        importance_type='split',
                                                        learning_rate=0.1,
                                                        max_depth=-1,
                                                        min_child_samples=20,
                                                        min_child_weig

In [19]:
dt = create_model('dt', fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,7.7355,131.5468,11.4694,0.3413,0.2988,0.2093
1,7.2573,115.3861,10.7418,0.3920,0.2837,0.2101
2,7.1842,109.4192,10.4604,0.4770,0.2832,0.2058
3,6.6541,97.7258,9.8856,0.5237,0.2589,0.1815
4,6.8579,95.7909,9.7873,0.5177,0.2377,0.1739
Mean,7.1378,109.9738,10.4689,0.4503,0.2724,0.1961
SD,0.3704,13.0089,0.6132,0.0720,0.0216,0.0153


In [20]:
rf = create_model('rf', round = 2)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,6.58,90.23,9.50,0.56,0.24,0.18
1,6.35,75.96,8.72,0.61,0.24,0.18
2,6.34,85.74,9.26,0.58,0.25,0.19
3,5.91,63.86,7.99,0.64,0.22,0.17
4,5.90,68.89,8.30,0.68,0.21,0.17
5,6.36,77.59,8.81,0.62,0.23,0.18
6,5.26,56.67,7.53,0.66,0.21,0.16
7,6.32,91.82,9.58,0.62,0.23,0.16
8,6.00,69.54,8.34,0.64,0.19,0.14
9,6.33,73.77,8.59,0.64,0.22,0.17


In [22]:
lightgbm = create_model('lightgbm', round = 2)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,6.59,91.36,9.56,0.55,0.23,0.17
1,6.32,74.72,8.64,0.62,0.23,0.18
2,6.16,73.38,8.57,0.64,0.23,0.18
3,5.79,61.91,7.87,0.65,0.21,0.17
4,5.94,68.32,8.27,0.68,0.22,0.17
5,6.41,72.29,8.50,0.65,0.23,0.18
6,5.58,57.00,7.55,0.66,0.21,0.16
7,6.31,81.66,9.04,0.66,0.21,0.17
8,6.01,68.44,8.27,0.65,0.19,0.14
9,6.54,78.23,8.84,0.61,0.22,0.18


In [24]:
tuned_lightgbm = tune_model(lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,6.8090,95.0804,9.7509,0.5338,0.2390,0.1828
1,6.4660,75.2544,8.6749,0.6148,0.2329,0.1836
2,6.5015,82.6292,9.0901,0.5924,0.2466,0.1938
3,6.1683,69.8858,8.3598,0.6040,0.2349,0.1856
4,6.0535,73.1129,8.5506,0.6580,0.2230,0.1707
5,6.7742,81.3648,9.0202,0.6020,0.2438,0.1968
6,6.0102,65.7396,8.1080,0.6097,0.2398,0.1881
7,6.7551,94.4624,9.7192,0.6063,0.2290,0.1788
8,6.1603,72.5302,8.5165,0.6270,0.1903,0.1484
9,6.3786,76.3887,8.7401,0.6232,0.2276,0.1773


In [25]:
tuned_lightgbm2 = tune_model(lightgbm, n_iter = 50)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,6.6011,88.4871,9.4068,0.5661,0.2240,0.1700
1,6.4738,74.4887,8.6307,0.6187,0.2264,0.1760
2,6.4881,82.4476,9.0801,0.5933,0.2449,0.1902
3,5.9747,68.3473,8.2672,0.6127,0.2255,0.1743
4,6.0714,71.0705,8.4303,0.6676,0.2272,0.1771
5,6.7410,84.1803,9.1750,0.5882,0.2358,0.1873
6,6.2480,67.6702,8.2262,0.5983,0.2236,0.1811
7,6.6909,88.5356,9.4093,0.6310,0.2219,0.1768
8,6.4033,73.5264,8.5748,0.6219,0.2005,0.1555
9,6.6982,82.0689,9.0592,0.5952,0.2260,0.1809


## Probando con tabnet

In [26]:
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
np.random.seed(0)


import os
from pathlib import Path

In [27]:

train = dataset.copy()
target = 'TASADE'
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [28]:
train.describe()

,TASADE,DEUDA_ENE21,DEUDA_FEB21,DEUDA_MAR21,DEUDA_ABR21,DEUDA_MAY21,DEV_ENE21,DEV_FEB21,DEV_MAR21,DEV_ABR21,DEV_MAY21,var_saldo_ene_feb,var_saldo_feb_mar,var_saldo_mar_abr,var_saldo_abr_may
count,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000,3937.000000
mean,43.494515,8752.537841,8365.527140,7968.912687,7578.399502,7169.572855,105.626325,80.433774,90.634209,79.738491,77.850521,-387.010701,-396.614453,-390.513185,-408.826647
std,14.796194,18736.882844,18280.145972,17869.742704,17490.225148,17091.943681,145.890914,118.313094,132.594975,121.543642,122.350111,919.021259,777.526576,808.291364,690.652889
min,9.000000,136.000000,127.340000,123.270000,119.430000,1.920000,0.000000,0.000000,0.000000,0.000000,0.000000,-25797.780000,-25998.760000,-35086.430000,-20526.150000
25%,35.000000,2000.000000,1862.100000,1714.300000,1567.350000,1411.920000,34.080000,23.640000,26.950000,22.450000,20.600000,-376.420000,-404.560000,-396.030000,-426.220000
50%,43.500000,3500.000000,3264.170000,3002.140000,2745.100000,2483.320000,62.570000,45.500000,50.860000,43.510000,40.890000,-208.090000,-225.130000,-221.960000,-233.020000
75%,50.500000,8000.000000,7529.740000,6931.370000,6471.360000,6114.260000,119.420000,89.860000,100.910000,87.890000,84.660000,-128.440000,-144.680000,-142.760000,-151.090000
max,89.000000,365000.000000,359657.540000,353203.270000,347006.210000,334341.350000,2441.750000,1697.030000,1847.430000,1708.060000,1704.210000,-0.010000,-0.010000,-0.010000,-0.010000


In [29]:
categorical_columns = []
categorical_dims =  {}
for col in train.columns[train.dtypes == object]:
    print(col, train[col].nunique())
    l_enc = LabelEncoder()
    train[col] = train[col].fillna("VV_likely")
    train[col] = l_enc.fit_transform(train[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

for col in train.columns[train.dtypes == 'float64']:
    train.fillna(train.loc[train_indices, col].mean(), inplace=True)

Set 3


In [30]:

unused_feat = ['Set']

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

# define your embedding sizes : here just a random choice
cat_emb_dim = [5, 4, 3, 6, 2, 2, 1, 10]

In [31]:
clf = TabNetRegressor(cat_dims=cat_dims, cat_emb_dim=cat_emb_dim, cat_idxs=cat_idxs)


Device used : cpu


In [32]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices].reshape(-1, 1)

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices].reshape(-1, 1)

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices].reshape(-1, 1)

In [33]:
max_epochs = 1000 if not os.getenv("CI", False) else 2


In [37]:

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
    max_epochs=max_epochs,
    patience=50,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

epoch 0  | loss: 92.35481| train_rmsle: 0.07662 | train_mae: 8.8879  | train_rmse: 11.93374| train_mse: 142.41413| valid_rmsle: 0.09023 | valid_mae: 9.33368 | valid_rmse: 12.56792| valid_mse: 157.95263|  0:00:00s
epoch 1  | loss: 86.48171| train_rmsle: 0.0694  | train_mae: 7.81487 | train_rmse: 11.00283| train_mse: 121.06237| valid_rmsle: 0.08689 | valid_mae: 8.30986 | valid_rmse: 11.75748| valid_mse: 138.23844|  0:00:00s
epoch 2  | loss: 90.35755| train_rmsle: 0.07354 | train_mae: 8.16374 | train_rmse: 11.40384| train_mse: 130.04764| valid_rmsle: 0.08923 | valid_mae: 8.62448 | valid_rmse: 12.0974 | valid_mse: 146.34702|  0:00:01s
epoch 3  | loss: 91.66994| train_rmsle: 0.06715 | train_mae: 7.46629 | train_rmse: 10.59201| train_mse: 112.19062| valid_rmsle: 0.08357 | valid_mae: 7.89617 | valid_rmse: 11.31712| valid_mse: 128.07713|  0:00:02s
epoch 4  | loss: 87.70627| train_rmsle: 0.06692 | train_mae: 7.73358 | train_rmse: 10.80241| train_mse: 116.69203| valid_rmsle: 0.08268 | valid_mae:

In [28]:
dataset_name='teas aprox'

In [29]:
# Deprecated : best model is automatically loaded at end of fit
# clf.load_best_model()

preds = clf.predict(X_test)

y_true = y_test

test_score = mean_squared_error(y_pred=preds, y_true=y_true)

print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_score}")

BEST VALID SCORE FOR teas aprox : 69.06783894669711
FINAL TEST SCORE FOR teas aprox : 88.39034908771006


In [30]:
clf.feature_importances_

array([3.14764681e-01, 8.43611091e-02, 5.11797623e-02, 9.18162542e-02,
       1.53563519e-05, 1.21185189e-01, 4.50448095e-02, 1.78444844e-01,
       1.96208117e-05, 1.10811755e-01, 6.62132207e-06, 1.49340392e-05,
       0.00000000e+00, 2.33506361e-03])

In [38]:
y_true[10:20]

array([[53.  ],
       [42.58],
       [53.1 ],
       [25.  ],
       [14.7 ],
       [44.  ],
       [55.  ],
       [60.01],
       [53.  ],
       [58.1 ]])

In [37]:
preds[10:20]

array([[47.520065],
       [49.565594],
       [44.806843],
       [21.192265],
       [24.578106],
       [40.132202],
       [56.820724],
       [50.10488 ],
       [42.26044 ],
       [52.042847]], dtype=float32)